In [1]:
# 기본 패키지 불러오기 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns 
plt.style.use("seaborn")
sns.set(font_scale=1)
sns.set_style("whitegrid")

# 그래프를 노트북 안에 그리기 위해 설정
%matplotlib inline

# 맥 OS 폰트 깨짐 방지 코드
from matplotlib import rc
import matplotlib as mpl 
import matplotlib.font_manager as fm 

rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

# 가격 컬럼과의 관계분석을 위한 전처리 진행 (Inside Airbnb: London Data)

# 데이터 준비

### 데이터 불러오기

In [2]:
# 데이터 불러오기
london_lists_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/listings.csv')
london_lists_summary_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/listings_outline.csv')
london_neighbourhoods_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/neighbourhoods.csv')
london_reviews_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/reviews.csv')
london_reviews_summary_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/reviews_outline.csv')

# 분석할 df 별도 복사
london_lists = london_lists_raw.copy()
london_reviews = london_reviews_raw.copy()

# 불러온 df 확인
london_lists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90852 entries, 0 to 90851
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            90852 non-null  int64  
 1   listing_url                                   90852 non-null  object 
 2   scrape_id                                     90852 non-null  int64  
 3   last_scraped                                  90852 non-null  object 
 4   source                                        90852 non-null  object 
 5   name                                          90852 non-null  object 
 6   description                                   87851 non-null  object 
 7   neighborhood_overview                         47521 non-null  object 
 8   picture_url                                   90842 non-null  object 
 9   host_id                                       90852 non-null 

### 데이터 불러온 최종일자 확인 

In [3]:
london_lists_raw['last_scraped'].unique()

array(['2024-03-21', '2024-03-20', '2024-03-22', '2024-03-19'],
      dtype=object)

슈퍼호스트 선정 기준 날짜 년 4회(분기별)
- 1월 1일 / 4월 1일 / 7월 1일 / 10월 1일 

### 분석에서 사용하지 않을 컬럼 삭제

In [4]:
# 1차 선별한 컬럼 25개 중 22개 2차 선별 + 1개(숙소_예약가능_여부)
# 최근 리뷰가 작성되었는지를 보기 위해 2개 컬럼 추가_240529 >> 'number_of_reviews_ltm' // 'last_review'
# URL 컬럼 london_lists에 적용_240530

columns_selected = ['id', 'listing_url', 'host_id', 'host_is_superhost', 'neighbourhood_cleansed', 'property_type', \
    'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'amenities', 'price', 'has_availability', 'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'last_review', 'review_scores_rating', 'review_scores_accuracy', \
    'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']

# 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_total_listings_count', 'host_has_profile_pic', 'host_identity_verified', 'minimum_nights', 'maximum_nights', 'has_availability',

# 선별한 컬럼만 적용
london_lists = london_lists[columns_selected]

# url도 포함한 df (실제 에어비앤비 사이트 들어가서 확인할 용도)
columns_selected_url = ['id', 'listing_url', 'host_id', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_total_listings_count', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood_cleansed', \
    'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'has_availability', 'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'last_review', 'review_scores_rating', 'review_scores_accuracy', \
    'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']

london_url = london_lists_raw[columns_selected_url]

# 변수 정리 
london_lists = london_lists.rename(columns={
    'id': '숙소_id',
    'listing_url' : '숙소_url',
    'host_id': '호스트_id',
    # 'host_response_time': '답변_평균시간',
    # 'host_response_rate': '문의_응답률',
    # 'host_acceptance_rate': '예약_수락률',
    'host_is_superhost': '슈퍼호스트',
    # 'host_total_listings_count': '숙소_수',
    # 'host_has_profile_pic' : '프로필_사진',
    # 'host_identity_verified' : '호스트_신원',
    'neighbourhood_cleansed': '숙소_지역',
    'property_type' : '숙소_특징',
    'room_type': '숙소_유형',
    'accommodates': '수용_인원수',
    'bathrooms': '욕실수',
    'bedrooms': '침실수',
    'beds': '침대수',
    'amenities': '편의시설',
    'price': '숙소_가격',
    # 'minimum_nights': '최소_숙박일',
    # 'maximum_nights': '최대_숙박일',
    'has_availability' : '예약_가능여부',
    'number_of_reviews': '리뷰수',
    'number_of_reviews_ltm' : '12개월_리뷰수',
    'number_of_reviews_l30d': '30일_리뷰수',
    'last_review' : '마지막_리뷰',
    'review_scores_rating': '리뷰점수',
    'review_scores_accuracy': '숙소_정확성_리뷰점수',
    'review_scores_cleanliness': '숙소_청결도_리뷰점수',
    'review_scores_checkin': '숙소_체크인_리뷰점수',
    'review_scores_communication': '숙소_소통_리뷰점수',
    'review_scores_location': '숙소_위치_리뷰점수',
    'review_scores_value': '숙소_가격_리뷰점수',
    'reviews_per_month': '평균_리뷰수'
 
})

# url 포함 df 변수 정리 
london_url = london_url.rename(columns={
    'id': '숙소_id',
    'listing_url' : '숙소_url',
    'host_id': '호스트_id',
    'host_response_time': '답변_평균시간',
    'host_response_rate': '문의_응답률',
    'host_acceptance_rate': '예약_수락률',
    'host_is_superhost': '슈퍼호스트',
    'host_total_listings_count': '숙소_수',
    'host_has_profile_pic' : '프로필_사진',
    'host_identity_verified' : '호스트_신원',
    'neighbourhood_cleansed': '숙소_지역',
    'property_type' : '숙소_특징',
    'room_type': '숙소_유형',
    'accommodates': '수용_인원수',
    'bathrooms': '욕실수',
    'bedrooms': '침실수',
    'beds': '침대수',
    'amenities': '편의시설',
    'price': '숙소_가격',
    'minimum_nights': '최소_숙박일',
    'maximum_nights': '최대_숙박일',
    'has_availability' : '숙소_예약_가능여부',
    'number_of_reviews': '리뷰수',
    'number_of_reviews_ltm' : '12개월_리뷰수',
    'number_of_reviews_l30d': '30일_리뷰수',
    'last_review' : '마지막_리뷰',
    'review_scores_rating': '리뷰점수',
    'review_scores_accuracy': '숙소_정확성_리뷰점수',
    'review_scores_cleanliness': '숙소_청결도_리뷰점수',
    'review_scores_checkin': '숙소_체크인_리뷰점수',
    'review_scores_communication': '숙소_소통_리뷰점수',
    'review_scores_location': '숙소_위치_리뷰점수',
    'review_scores_value': '숙소_가격_리뷰점수',
    'reviews_per_month': '평균_리뷰수'
})


# 컬럼명 확인
london_lists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90852 entries, 0 to 90851
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   숙소_id        90852 non-null  int64  
 1   숙소_url       90852 non-null  object 
 2   호스트_id       90852 non-null  int64  
 3   슈퍼호스트        90463 non-null  object 
 4   숙소_지역        90852 non-null  object 
 5   숙소_특징        90852 non-null  object 
 6   숙소_유형        90852 non-null  object 
 7   수용_인원수       90852 non-null  int64  
 8   욕실수          57894 non-null  float64
 9   침실수          78118 non-null  float64
 10  침대수          57837 non-null  float64
 11  편의시설         90852 non-null  object 
 12  숙소_가격        57885 non-null  object 
 13  예약_가능여부      86773 non-null  object 
 14  리뷰수          90852 non-null  int64  
 15  12개월_리뷰수     90852 non-null  int64  
 16  30일_리뷰수      90852 non-null  int64  
 17  마지막_리뷰       65869 non-null  object 
 18  리뷰점수         65869 non-null  float64
 19  숙소_정

### 결측치 처리에 앞서서 리뷰가 0인 컬럼 제거

In [5]:
# 리뷰수가 0인 컬럼 조건 
condition_review_0 = london_lists['리뷰수'] == 0

# 별도의 DF로 저장
london_lists_review_0 = london_lists[condition_review_0]

리뷰가 0인 데이터의 특징 파악

In [6]:
# 리뷰수가 0인 컬럼의 특징 파악_리뷰데이터 관련

# 개수: 24,983개
print('리뷰가 0개인 런던 내 숙소 개수: ')
print(london_lists_review_0.shape)
print()

# 리뷰가 0개인데 각 리뷰점수가 Null값이 아닌 숙소 개수 

score_type = ['리뷰점수', '숙소_정확성_리뷰점수', '숙소_청결도_리뷰점수', '숙소_체크인_리뷰점수', '숙소_소통_리뷰점수', '숙소_위치_리뷰점수', '숙소_가격_리뷰점수', '평균_리뷰수']

for type in score_type:
    print(f'{type} :')
    print(f'{type}가 null이 아닌 데이터의 개수 >> {london_lists_review_0[type].notnull().sum()}개')
    print()

리뷰가 0개인 런던 내 숙소 개수: 
(24983, 26)

리뷰점수 :
리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_정확성_리뷰점수 :
숙소_정확성_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_청결도_리뷰점수 :
숙소_청결도_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_체크인_리뷰점수 :
숙소_체크인_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_소통_리뷰점수 :
숙소_소통_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_위치_리뷰점수 :
숙소_위치_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_가격_리뷰점수 :
숙소_가격_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

평균_리뷰수 :
평균_리뷰수가 null이 아닌 데이터의 개수 >> 0개



In [7]:
# # 리뷰수가 0인 컬럼의 특징 파악_슈퍼호스트 관련

print(london_lists_review_0['슈퍼호스트'].unique())
print()
print('리뷰슈가 0개인 숙소 중 호스트가 슈퍼호스트인지 여부: ')
print(london_lists_review_0['슈퍼호스트'].value_counts())
print()
print('리뷰수가 0개인 숙소 데이터 중 슈퍼호스트가 null값인 데이터 개수:')
print(f'{london_lists_review_0["슈퍼호스트"].isnull().sum()}개')

['f' 't' nan]

리뷰슈가 0개인 숙소 중 호스트가 슈퍼호스트인지 여부: 
f    23754
t     1171
Name: 슈퍼호스트, dtype: int64

리뷰수가 0개인 숙소 데이터 중 슈퍼호스트가 null값인 데이터 개수:
58개


리뷰가 0개가 아닌 데이터만 사용

In [8]:
# 리뷰 개수가 0이 아닌 컬럼 조건 설정
condition_review = london_lists['리뷰수'] != 0

# 리뷰 개수가 0이 아닌 컬럼으로 df 업데이트 
london_lists = london_lists[condition_review]
london_url = london_url[condition_review]

# print(london_lists[:10])
# print()
print(london_lists.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65869 entries, 0 to 90735
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   숙소_id        65869 non-null  int64  
 1   숙소_url       65869 non-null  object 
 2   호스트_id       65869 non-null  int64  
 3   슈퍼호스트        65538 non-null  object 
 4   숙소_지역        65869 non-null  object 
 5   숙소_특징        65869 non-null  object 
 6   숙소_유형        65869 non-null  object 
 7   수용_인원수       65869 non-null  int64  
 8   욕실수          43758 non-null  float64
 9   침실수          58052 non-null  float64
 10  침대수          43703 non-null  float64
 11  편의시설         65869 non-null  object 
 12  숙소_가격        43728 non-null  object 
 13  예약_가능여부      64536 non-null  object 
 14  리뷰수          65869 non-null  int64  
 15  12개월_리뷰수     65869 non-null  int64  
 16  30일_리뷰수      65869 non-null  int64  
 17  마지막_리뷰       65869 non-null  object 
 18  리뷰점수         65869 non-null  float64
 19  숙소_정

### 숙소 유형 [Entire home/apt / Private room] 데이터만 선별

- 전체 데이터중 99.5% 비율 차지
1. Entire home/apt : 64.74%
2. Private Room: 34.7%

In [9]:
# 전체 데이터에서 각 숙소 유형별 개수 및 전체 대비 비율 계산하기

# 함수 선언

def CategoricalData_Count_Ratio(name_of_dataframe, name_of_column):

    print(f'전체 숙소 개수: {name_of_dataframe.shape[0]}개')
    print()

    type_list = name_of_dataframe[name_of_column].unique().tolist()

    for type in type_list:
        condition_type = name_of_dataframe[name_of_column] == type
        ratio_type = round((name_of_dataframe[condition_type].shape[0] / name_of_dataframe.shape[0]) * 100, 2)
        print(f'{name_of_column}이 {type}인 숙소의 개수: {name_of_dataframe[condition_type].shape[0]}개')
        print(f'{name_of_column}이 {type}인 숙소의 비율 : {ratio_type}%')
        print()

# 출력 
CategoricalData_Count_Ratio(london_lists, '숙소_유형')

전체 숙소 개수: 65869개

숙소_유형이 Entire home/apt인 숙소의 개수: 42642개
숙소_유형이 Entire home/apt인 숙소의 비율 : 64.74%

숙소_유형이 Private room인 숙소의 개수: 22857개
숙소_유형이 Private room인 숙소의 비율 : 34.7%

숙소_유형이 Hotel room인 숙소의 개수: 149개
숙소_유형이 Hotel room인 숙소의 비율 : 0.23%

숙소_유형이 Shared room인 숙소의 개수: 221개
숙소_유형이 Shared room인 숙소의 비율 : 0.34%



In [10]:
# 조건 설정
condition_room_entirehomeapt = london_lists['숙소_유형'] == 'Entire home/apt'
condition_room_privateroom = london_lists['숙소_유형'] == 'Private room'

# 데이터 제거
london_lists = london_lists[(condition_room_entirehomeapt | condition_room_privateroom)]
london_url = london_url[(condition_room_entirehomeapt | condition_room_privateroom)]

# 데이터 확인
CategoricalData_Count_Ratio(london_lists, '숙소_유형')

전체 숙소 개수: 65499개

숙소_유형이 Entire home/apt인 숙소의 개수: 42642개
숙소_유형이 Entire home/apt인 숙소의 비율 : 65.1%

숙소_유형이 Private room인 숙소의 개수: 22857개
숙소_유형이 Private room인 숙소의 비율 : 34.9%



### 예약_가능여부 Null값 제거 및 예약 불가능 숙소 제거
* null값 비율 : 2.02% >> 제거 

In [11]:
# 결측치 조회
print(f'예약_가능여부가 Null인 데이터 개수: {london_lists["예약_가능여부"].isnull().sum()}개')
print(f'예약_가능여부가 Null값인 데이터의 비율: {round((london_lists["예약_가능여부"].isnull().sum() / london_lists.shape[0]) * 100, 2)}%')

# 결측치 제거
london_lists = london_lists[london_lists["예약_가능여부"].notnull()]
print(f'Null값 제거 후 데이터 개수: {london_lists.shape[0]}개')

예약_가능여부가 Null인 데이터 개수: 1320개
예약_가능여부가 Null값인 데이터의 비율: 2.02%
Null값 제거 후 데이터 개수: 64179개


In [12]:
london_lists['예약_가능여부'].value_counts()

t    64110
f       69
Name: 예약_가능여부, dtype: int64

In [13]:
# 조건 생성
condition_availability_f = london_lists['예약_가능여부'] == 'f'

# 조회할 컬럼 지정 
col_lists = ['숙소_id', '호스트_id', '슈퍼호스트', '숙소_유형', '욕실수', '침실수', '침대수', '숙소_가격','리뷰수', '리뷰점수']

# 데이터 조회
london_lists[col_lists][condition_availability_f]

,숙소_id,호스트_id,슈퍼호스트,숙소_유형,욕실수,침실수,침대수,숙소_가격,리뷰수,리뷰점수
95,78606,422362,f,Private room,NaN,1.0,NaN,NaN,2,5.00
4314,6194112,2495450,f,Private room,NaN,NaN,NaN,NaN,74,4.66
4816,6788203,34064935,f,Private room,NaN,NaN,NaN,NaN,1,4.00
8229,10146851,28886982,f,Entire home/apt,2.0,2.0,2.0,$165.00,43,4.77
12112,14363227,88085116,f,Private room,NaN,NaN,NaN,NaN,2,5.00
...,...,...,...,...,...,...,...,...,...,...
56395,780392890725657904,491180264,f,Entire home/apt,1.0,2.0,2.0,$184.00,7,4.86
57411,799256485404304603,489825790,f,Entire home/apt,NaN,1.0,NaN,NaN,4,4.25
58563,817486735163198767,495903560,f,Entire home/apt,2.0,2.0,2.0,$350.00,1,5.00
67985,925856225370514485,134663346,f,Private room,1.0,1.0,3.0,$69.00,20,4.90


In [14]:
# 데이터 삭제 

london_lists = london_lists[~condition_availability_f]

# 결측치 처리

In [15]:
# 컬럼 전체 결측치 확인 

london_lists.isnull().sum()
# 전체 개수 : 65,449개 중 

숙소_id              0
숙소_url             0
호스트_id             0
슈퍼호스트            327
숙소_지역              0
숙소_특징              0
숙소_유형              0
수용_인원수             0
욕실수            20624
침실수             7075
침대수            20679
편의시설               0
숙소_가격          20654
예약_가능여부            0
리뷰수                0
12개월_리뷰수           0
30일_리뷰수            0
마지막_리뷰             0
리뷰점수               0
숙소_정확성_리뷰점수       30
숙소_청결도_리뷰점수       26
숙소_체크인_리뷰점수       54
숙소_소통_리뷰점수        34
숙소_위치_리뷰점수        54
숙소_가격_리뷰점수        56
평균_리뷰수             0
dtype: int64

## 가격이 null값인 컬럼과 가격이 null값이 아닌 컬럼으로 분리 
* 가격과 기타 컬럼 간의 관계 분석 목적 

In [16]:
# 가격이 null값/notnull값 조건 생성 
condition_price_notnull = london_lists['숙소_가격'].notnull()
condition_price_null = london_lists['숙소_가격'].isnull()

# 가격이 null값/null값이 아닌 df 분리
london_lists_price = london_lists[condition_price_notnull] 
london_lists_price_null = london_lists[condition_price_null]

# df 데이터 개수 확인 
print(london_lists_price.shape)
print(london_lists_price_null.shape)

# 변수명 정리 
london_lists = london_lists_price

(43456, 26)
(20654, 26)


In [17]:
london_lists.isnull().sum()

숙소_id            0
숙소_url           0
호스트_id           0
슈퍼호스트          266
숙소_지역            0
숙소_특징            0
숙소_유형            0
수용_인원수           0
욕실수             21
침실수             11
침대수             76
편의시설             0
숙소_가격            0
예약_가능여부          0
리뷰수              0
12개월_리뷰수         0
30일_리뷰수          0
마지막_리뷰           0
리뷰점수             0
숙소_정확성_리뷰점수     17
숙소_청결도_리뷰점수     18
숙소_체크인_리뷰점수     18
숙소_소통_리뷰점수      17
숙소_위치_리뷰점수      18
숙소_가격_리뷰점수      19
평균_리뷰수           0
dtype: int64

## 슈퍼호스트 결측치 처리
* 우선 결측치 제거하고 진행

컬럼 전처리 상황
* 리뷰수가 0인 컬럼 제거
* 숙소 유형 [Entire home/apt / Private room] 데이터만 선별
* 가격이 null값인 컬럼 제거한 상태 

슈퍼호스트 기준
1. 숙박 10건 이상 호스팅 또는 3건의 예약에 걸쳐 총 100박 이상 호스팅
2. 응답률 90% 이상 유지
3. 예약 취소율 1% 미만 유지. 단, 정상참작이 가능한 상황에 따른 예약 취소는 제외
4. 전체 평점 4.8점 이상 유지(후기 작성 기한인 14일이 지나거나, 그전에라도 게스트와 호스트 양측이 모두 후기를 제출하면 후기는 슈퍼호스트 실적에 계산됩니다).

In [18]:
# 슈퍼호스트 null 값 : 266개
print(f'슈퍼호스트 null값 개수: {london_lists["슈퍼호스트"].isnull().sum()}개')

# 슈퍼호스트 null값의 비율: 0.61% >> 삭제해도 무방 
print(f'슈퍼호스트 null값의 비율: {round((london_lists["슈퍼호스트"].isnull().sum() / london_lists.shape[0]) * 100, 2)}%')

슈퍼호스트 null값 개수: 266개
슈퍼호스트 null값의 비율: 0.61%


In [19]:
# 리뷰가 0개인 숙소를 모아놓은 데이터프레임에도 적용 

# 슈퍼호스트 null 값 : 58개
print(f'슈퍼호스트 null값 개수: {london_lists_review_0["슈퍼호스트"].isnull().sum()}개')

# 슈퍼호스트 null값의 비율: 0.23% >> 삭제해도 무방 
print(f'슈퍼호스트 null값의 비율: {round((london_lists_review_0["슈퍼호스트"].isnull().sum() / london_lists_review_0.shape[0]) * 100, 2)}%')

슈퍼호스트 null값 개수: 58개
슈퍼호스트 null값의 비율: 0.23%


In [20]:
london_lists['슈퍼호스트'].value_counts()

f    31888
t    11302
Name: 슈퍼호스트, dtype: int64

슈퍼호스트 컬럼 / 전처리 방향 설정 결론 (슈퍼호스트_런던_240524.ipynb 파일 참고)

- null 값을 삭제해도 무방하다. (비율이 0.6%)
- 결측치를 일괄 대치하려면 최빈값으로 대치하는 것이 맞을듯
- 결측치를 실제 데이터와 비교해보는 방법도 있다. 
    - NGD 기법 또는 크롤링 

In [21]:
# 슈퍼호스트 결측치 제거
print(f'삭제 전 전체 데이터 개수: {london_lists.shape[0]}개')
print(f"삭제 할 슈퍼호스트 null값 데이터 개수: {london_lists['슈퍼호스트'].isnull().sum()}개")
london_lists = london_lists.dropna(subset=['슈퍼호스트'])
print()

# 제거 후 확인 
print(f"제거 후 슈퍼호스트 컬럼의 null값 확인: {london_lists['슈퍼호스트'].isnull().sum()}개")
print(f'Null값 제거 후 전체 데이터 개수 : {london_lists.shape[0]}개')

삭제 전 전체 데이터 개수: 43456개
삭제 할 슈퍼호스트 null값 데이터 개수: 266개

제거 후 슈퍼호스트 컬럼의 null값 확인: 0개
Null값 제거 후 전체 데이터 개수 : 43190개


## 욕실수 / 침실수 / 침대수 결측치 처리 
* 결론: 우선 결측치 제거하고 진행 

현재 데이터 전처리 현황 
* 리뷰수가 0개인 컬럼 제거 
* 가격이 Null값이 아닌 컬럼만 선택 
* 슈퍼호스트가 Null값인 데이터 제거 

In [22]:
# 결측치 수치 확인 
london_lists.isnull().sum()

숙소_id           0
숙소_url          0
호스트_id          0
슈퍼호스트           0
숙소_지역           0
숙소_특징           0
숙소_유형           0
수용_인원수          0
욕실수            21
침실수            11
침대수            76
편의시설            0
숙소_가격           0
예약_가능여부         0
리뷰수             0
12개월_리뷰수        0
30일_리뷰수         0
마지막_리뷰          0
리뷰점수            0
숙소_정확성_리뷰점수    17
숙소_청결도_리뷰점수    18
숙소_체크인_리뷰점수    18
숙소_소통_리뷰점수     17
숙소_위치_리뷰점수     18
숙소_가격_리뷰점수     19
평균_리뷰수          0
dtype: int64

#### 결측치 확인

In [23]:
# 욕실수, 침실수, 침대수가 공통적으로 Null값인 컬럼 확인 

facility_null_list = [['욕실수', '침실수', '침대수'], ['욕실수', '침대수'], ['욕실수', '침실수'], ['침대수', '침실수']]

for facility in facility_null_list:
    condition_facilities_null = london_lists[facility].isnull().all(axis=1)
    print(f'{facility}가 모두 null값인 데이터의 개수: {london_lists[condition_facilities_null].shape[0]}개')

['욕실수', '침실수', '침대수']가 모두 null값인 데이터의 개수: 0개
['욕실수', '침대수']가 모두 null값인 데이터의 개수: 1개
['욕실수', '침실수']가 모두 null값인 데이터의 개수: 0개
['침대수', '침실수']가 모두 null값인 데이터의 개수: 0개


In [24]:
# 전체 데이터 대비 ('욕실수', '침실수', '침대수' 컬럼의 결측치 합 개수) 비율 확인 : 0.25%

facilities = ['욕실수', '침실수', '침대수']

sum_facilities_null = 0

for f in facilities:
    sum = london_lists[f].isnull().sum()
    sum_facilities_null += sum
    
ratio_facilities_null = round(((sum_facilities_null - london_lists[["욕실수", "침대수"]].isnull().all(axis=1).sum()) / london_lists.shape[0]) * 100, 2)
print(f'욕실수, 침대수, 침실수의 null값 합계 : {(sum_facilities_null - london_lists[["욕실수", "침대수"]].isnull().all(axis=1).sum())}개')
print(f'전체 데이터 대비 숙소 시설 결측치 비율 : {ratio_facilities_null}%')

욕실수, 침대수, 침실수의 null값 합계 : 107개
전체 데이터 대비 숙소 시설 결측치 비율 : 0.25%


#### 제거 가능 하지만 크롤링한 데이터 비교해보자 
* 침대수, 침실수, 욕실수 데이터는 시간의 영향을 매우 적은 데이터 

In [25]:
fac1 = pd.read_csv('../../Data/침실_침대_욕실수/London_침실대욕_1.csv', encoding='cp949')
fac2 = pd.read_csv('../../Data/침실_침대_욕실수/London_침실대욕_2.csv', encoding='cp949')
fac3 = pd.read_csv('../../Data/침실_침대_욕실수/London_침실대욕_3.csv', encoding='cp949')
fac4 = pd.read_csv('../../Data/침실_침대_욕실수/London_침실대욕_4.csv', encoding='cp949')

crawl_fac_null = pd.concat([fac1, fac2, fac3, fac4], axis=0, ignore_index=True)
print(crawl_fac_null.shape)
print(crawl_fac_null.info())

(4000, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   호스트_id  1000 non-null   float64
 1   숙소_url  4000 non-null   object 
 2   etc     3208 non-null   object 
 3   숙소_id   3000 non-null   float64
dtypes: float64(2), object(2)
memory usage: 125.1+ KB
None


In [26]:
crawl_fac_null[:10]

,호스트_id,숙소_url,etc,숙소_id
0,170702.0,https://www.airbnb.com/rooms/170702,침대 1개 · 공용 욕실,NaN
1,314985.0,https://www.airbnb.com/rooms/314985,침대 1개 · 욕실 없음,NaN
2,315658.0,https://www.airbnb.com/rooms/315658,더블 침대 1개 · 공용 욕실,NaN
3,39387.0,https://www.airbnb.com/rooms/39387,소파 베드 1개 · 공용 욕실,NaN
4,41509.0,https://www.airbnb.com/rooms/41509,침대 3개 · 공용 욕실,NaN
5,46992.0,https://www.airbnb.com/rooms/46992,퀸사이즈 침대 1개 · 침실에 딸린 전용 욕실,NaN
6,330044.0,https://www.airbnb.com/rooms/330044,퀸사이즈 침대 1개 · 전용 욕실,NaN
7,229684.0,https://www.airbnb.com/rooms/229684,침대 1개 · 공용 욕실,NaN
8,72913.0,https://www.airbnb.com/rooms/72913,더블 침대 1개 · 침실에 딸린 전용 욕실,NaN
9,341143.0,https://www.airbnb.com/rooms/341143,침대 1개 · 공용 욕실,NaN


당장 사용하긴 어려울 듯. 일단 제거하고 추후에 다시 집어넣자

#### 결측치 제거

In [27]:
# 결측치 제거 
print(f'삭제 전 전체 데이터 개수: {london_lists.shape[0]}개')
print(f"""삭제 할 슈퍼호스트 null값 데이터 개수: 
{london_lists[['욕실수', '침실수', '침대수']].isnull().sum()}""")
london_lists = london_lists.dropna(subset=['욕실수', '침실수', '침대수'])
print()

# 제거 완료됐나 확인
print(f"""남아있는 욕실수, 침실수, 침대수 null값 데이터 개수:  
{london_lists[['욕실수', '침실수', '침대수']].isnull().sum()}""")
print(f'Null값 제거 후 전체 데이터 개수: {london_lists.shape[0]}개')

삭제 전 전체 데이터 개수: 43190개
삭제 할 슈퍼호스트 null값 데이터 개수: 
욕실수    21
침실수    11
침대수    76
dtype: int64

남아있는 욕실수, 침실수, 침대수 null값 데이터 개수:  
욕실수    0
침실수    0
침대수    0
dtype: int64
Null값 제거 후 전체 데이터 개수: 43083개


## 리뷰 점수 컬럼 결측치 처리 


현재 데이터 전처리 현황 
* 리뷰수가 0개인 컬럼 제거 
* 가격이 Null값이 아닌 컬럼만 선택 
* 슈퍼호스트가 Null값인 데이터 제거 
* 욕실수 / 침실수 / 침대수 결측치 제거

In [28]:
# 결측치 확인
london_lists.isnull().sum()

숙소_id           0
숙소_url          0
호스트_id          0
슈퍼호스트           0
숙소_지역           0
숙소_특징           0
숙소_유형           0
수용_인원수          0
욕실수             0
침실수             0
침대수             0
편의시설            0
숙소_가격           0
예약_가능여부         0
리뷰수             0
12개월_리뷰수        0
30일_리뷰수         0
마지막_리뷰          0
리뷰점수            0
숙소_정확성_리뷰점수    16
숙소_청결도_리뷰점수    17
숙소_체크인_리뷰점수    17
숙소_소통_리뷰점수     16
숙소_위치_리뷰점수     17
숙소_가격_리뷰점수     18
평균_리뷰수          0
dtype: int64

In [29]:
# 각 리뷰 점수 컬럼이 모두 NULL 값인 데이터 개수 확인 : 15개
type_review_null_all = london_lists[['숙소_정확성_리뷰점수', '숙소_청결도_리뷰점수', '숙소_체크인_리뷰점수', '숙소_소통_리뷰점수', '숙소_위치_리뷰점수', '숙소_가격_리뷰점수']].isnull().all(axis=1)
print(f'각 기준별 리뷰점수가 모두 Null값인 데이터 개수: {london_lists[type_review_null_all].shape[0]}개')
print()

# 데이터 비율 확인 : 0.03%
print(f"전체 데이터 개수: {london_lists.shape[0]}개")
print(f"각 기준별 리뷰점수가 모두 NULL값인 데이터의 비율: {round((london_lists[type_review_null_all].shape[0] / london_lists.shape[0]) * 100, 2)}%")
print()

# 모두 NULL값인 데이터 제거
london_lists = london_lists[~type_review_null_all]
print(f"NULL값 제거 후 전체 데이터 개수: {london_lists.shape[0]}개")

각 기준별 리뷰점수가 모두 Null값인 데이터 개수: 15개

전체 데이터 개수: 43083개
각 기준별 리뷰점수가 모두 NULL값인 데이터의 비율: 0.03%

NULL값 제거 후 전체 데이터 개수: 43068개


In [30]:
# 각 리뷰 점수 컬럼별로 하나라도 NULL 값을 가지고 있는 데이터 개수 확인: 7개
type_review_null_any = london_lists[['숙소_정확성_리뷰점수', '숙소_청결도_리뷰점수', '숙소_체크인_리뷰점수', '숙소_소통_리뷰점수', '숙소_위치_리뷰점수', '숙소_가격_리뷰점수']].isnull().any(axis=1)
print(f'각 기준별 리뷰점수가 하나라도 Null값인 데이터 개수: {london_lists[type_review_null_any].shape[0]}개')
print()

# 데이터 비율 확인 : 0.02%
print(f"제거 전 전체 데이터 개수: {london_lists.shape[0]}개")
print(f"각 기준별 리뷰점수 중 하나라도 NULL값인 데이터의 비율: {round((london_lists[type_review_null_any].shape[0] / london_lists.shape[0]) * 100, 2)}%")
print()

# 하나라도 NULL값인 데이터 제거
london_lists = london_lists[~type_review_null_any]
print(f"NULL값 제거 후 전체 데이터 개수: {london_lists.shape[0]}개")

각 기준별 리뷰점수가 하나라도 Null값인 데이터 개수: 7개

제거 전 전체 데이터 개수: 43068개
각 기준별 리뷰점수 중 하나라도 NULL값인 데이터의 비율: 0.02%

NULL값 제거 후 전체 데이터 개수: 43061개


# 결측치 처리 후 데이터 확인 및 추가 전처리 

현재 데이터 전처리 현황 
* 리뷰수가 0개인 컬럼 제거 
* 숙소 유형 [Entire home/apt / Private room] 데이터만 선별
* 가격이 Null값이 아닌 컬럼만 선택 
* 슈퍼호스트가 Null값인 데이터 제거 
* 욕실수 / 침실수 / 침대수 결측치 제거
* 각 리뷰 점수 컬럼 결측치 제거 

기초 통계 요약 확인 / 이상치 확인 후 전처리해야할 컬럼 
* '욕실수', '침실수', '침대수', '숙소_가격', '리뷰수', '평균_리뷰수'

기초 통계 요약 확인
* '리뷰점수', '숙소_정확성_리뷰점수', '숙소_청결도_리뷰점수', '숙소_체크인_리뷰점수', '숙소_소통_리뷰점수', '숙소_위치_리뷰점수', '숙소_가격_리뷰점수'

## 런던 데이터 확인 

In [31]:
london_lists.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43061 entries, 0 to 90735
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   숙소_id        43061 non-null  int64  
 1   숙소_url       43061 non-null  object 
 2   호스트_id       43061 non-null  int64  
 3   슈퍼호스트        43061 non-null  object 
 4   숙소_지역        43061 non-null  object 
 5   숙소_특징        43061 non-null  object 
 6   숙소_유형        43061 non-null  object 
 7   수용_인원수       43061 non-null  int64  
 8   욕실수          43061 non-null  float64
 9   침실수          43061 non-null  float64
 10  침대수          43061 non-null  float64
 11  편의시설         43061 non-null  object 
 12  숙소_가격        43061 non-null  object 
 13  예약_가능여부      43061 non-null  object 
 14  리뷰수          43061 non-null  int64  
 15  12개월_리뷰수     43061 non-null  int64  
 16  30일_리뷰수      43061 non-null  int64  
 17  마지막_리뷰       43061 non-null  object 
 18  리뷰점수         43061 non-null  float64
 19  숙소_정

In [32]:
london_lists.isnull().sum()

숙소_id          0
숙소_url         0
호스트_id         0
슈퍼호스트          0
숙소_지역          0
숙소_특징          0
숙소_유형          0
수용_인원수         0
욕실수            0
침실수            0
침대수            0
편의시설           0
숙소_가격          0
예약_가능여부        0
리뷰수            0
12개월_리뷰수       0
30일_리뷰수        0
마지막_리뷰         0
리뷰점수           0
숙소_정확성_리뷰점수    0
숙소_청결도_리뷰점수    0
숙소_체크인_리뷰점수    0
숙소_소통_리뷰점수     0
숙소_위치_리뷰점수     0
숙소_가격_리뷰점수     0
평균_리뷰수         0
dtype: int64

In [33]:
london_lists[['욕실수', '침실수', '침대수', '숙소_가격', '리뷰수', '평균_리뷰수']].describe()

,욕실수,침실수,침대수,리뷰수,평균_리뷰수
count,43061.000000,43061.000000,43061.000000,43061.000000,43061.000000
mean,1.348529,1.494276,1.841829,29.148278,1.174376
std,0.624535,0.934127,1.348217,54.612105,1.302139
min,0.000000,0.000000,0.000000,1.000000,0.010000
25%,1.000000,1.000000,1.000000,3.000000,0.340000
50%,1.000000,1.000000,1.000000,11.000000,0.760000
75%,1.500000,2.000000,2.000000,31.000000,1.520000
max,13.000000,22.000000,38.000000,1457.000000,32.260000


리뷰수와 평균 리뷰수는 슈퍼호스트를 기준으로 나눠서 다시 파악해보자. 한꺼번에 보는 것은 맞지 않는 것 같다.

## 숙소_특징 

In [34]:
print(f'숙소_특징 컬럼의 Nunique 값: {london_lists["숙소_특징"].nunique()}개')
print('숙소 특징 컬럼의 Unique 값 :', london_lists['숙소_특징'].unique() )

숙소_특징 컬럼의 Nunique 값: 60개
숙소 특징 컬럼의 Unique 값 : ['Entire rental unit' 'Private room in rental unit' 'Entire home'
 'Private room in home' 'Entire townhouse' 'Entire guesthouse'
 'Private room in townhouse' 'Private room in condo' 'Entire condo'
 'Room in aparthotel' 'Entire serviced apartment' 'Private room in loft'
 'Private room' 'Private room in bed and breakfast' 'Entire guest suite'
 'Houseboat' 'Private room in serviced apartment'
 'Private room in guesthouse' 'Entire loft' 'Entire cabin'
 'Private room in bungalow' 'Private room in guest suite' 'Entire cottage'
 'Private room in villa' 'Private room in cottage' 'Tiny home'
 'Private room in houseboat' 'Private room in casa particular'
 'Shepherd’s hut' 'Private room in vacation home' 'Entire place'
 'Room in boutique hotel' 'Camper/RV' 'Entire vacation home' 'Boat'
 'Entire bungalow' 'Earthen home' 'Private room in hostel'
 'Private room in cabin' 'Entire villa' 'Entire home/apt'
 'Private room in hut' 'Room in hotel' 'Private roo

 숙소_특징 데이터 확인 (by 예진님)
- 'Entire rental unit': '전체 대여 유닛',
- 'Private room in rental unit': '대여 유닛 내 개인 방',
- 'Entire townhouse': '전체 타운하우스',
-    'Entire home': '전체 집',
-    'Private room in home': '집 내 개인 방',
-    'Entire guesthouse': '전체 게스트하우스',
-    'Private room in townhouse': '타운하우스 내 개인 방',
-   'Entire condo': '전체 콘도',
-    'Private room in condo': '콘도 내 개인 방',
-    'Room in aparthotel': '아파트호텔 내 방',
-    'Entire serviced apartment': '전체 서비스 아파트',
-    'Private room in loft': '로프트 내 개인 방',
-    'Private room': '개인 방',
-    'Private room in bed and breakfast': 'B&B 내 개인 방',
-    'Entire guest suite': '전체 게스트 스위트',
-    'Private room in serviced apartment': '서비스 아파트 내 개인 방',
-    'Private room in bungalow': '방갈로 내 개인 방',
-    'Private room in guesthouse': '게스트하우스 내 개인 방',
-   'Entire loft': '전체 로프트',
-    'Entire cabin': '전체 오두막',
-    'Private room in guest suite': '게스트 스위트 내 개인 방',
-    'Entire cottage': '전체 코티지',
-    'Private room in villa': '빌라 내 개인 방',
-   'Private room in cottage': '코티지 내 개인 방',
-  'Tiny home': '작은 집',
-    'Entire place': '전체 장소',,
-    'Private room in casa particular': '카사 파르티쿨라 내 개인 방',
-    'Shepherd’s hut': '바퀴달린 오두막',
-    'Private room in vacation home': '휴가용 주택 내 개인 방',
-    'Room in boutique hotel': '부티크 호텔 내 방',
-    'Camper/RV': '캠퍼/RV',
-    'Entire vacation home': '전체 휴가용 주택',
-    'Entire bungalow': '전체 방갈로',
-    'Private room in hut': '오두막 내 개인 방',
-    'Entire villa': '전체 빌라',
-    'Floor': '층',
-    'Earthen home': '흙집',
-    'Private room in hostel': '호스텔 내 개인 방',
-    'Private room in floor': '층 내 개인 방',
-    'Private room in cabin': '오두막 내 개인 방',
-    'Private room in island': '섬 내 개인 방',
-    'Entire home/apt': '전체 집/아파트',
-    'Room in hotel': '호텔 내 방',
-    'Private room in tiny home': '작은 집 내 개인 방',
-    'Private room in farm stay': '농장 체류 내 개인 방',
-    'Campsite': '캠프장',
-    'Barn': '헛간',
-    'Religious building': '종교 건물',
-    'Private room in earthen home': '흙집 내 개인 방',
-    'Private room in treehouse': '트리하우스 내 개인 방',
-    'Hut': '오두막',
-    'Entire chalet': '전체 샬레',
-    'Tent': '텐트',
-    'Private room in religious building': '종교 건물 내 개인 방',
-   'Minsu': '민수',
-    'Casa particular': '개인 홈스테이',
-    'Island': '섬',
-    'Castle': '성',
-    'Tower': '탑',
-    "Private room in shepherd's hut": '오두막 내 개인 방',
-    'Shipping container': '선적 컨테이너',
-    'Treehouse': '트리하우스'

-    Houseboat': '하우스보트',
-    'Private room in houseboat': '하우스보트 내 개인 방'
-    'Private room in yurt': '요트 내 개인 방',
-    'Boat': '보트',
-    'Private room in boat': '보트 내 개인 방',

In [35]:
# 숙소_특징 유니크 값별 개수 및 비율 확인

CategoricalData_Count_Ratio(london_lists, '숙소_특징')

전체 숙소 개수: 43061개

숙소_특징이 Entire rental unit인 숙소의 개수: 16943개
숙소_특징이 Entire rental unit인 숙소의 비율 : 39.35%

숙소_특징이 Private room in rental unit인 숙소의 개수: 4845개
숙소_특징이 Private room in rental unit인 숙소의 비율 : 11.25%

숙소_특징이 Entire home인 숙소의 개수: 3617개
숙소_특징이 Entire home인 숙소의 비율 : 8.4%

숙소_특징이 Private room in home인 숙소의 개수: 5273개
숙소_특징이 Private room in home인 숙소의 비율 : 12.25%

숙소_특징이 Entire townhouse인 숙소의 개수: 611개
숙소_특징이 Entire townhouse인 숙소의 비율 : 1.42%

숙소_특징이 Entire guesthouse인 숙소의 개수: 146개
숙소_특징이 Entire guesthouse인 숙소의 비율 : 0.34%

숙소_특징이 Private room in townhouse인 숙소의 개수: 711개
숙소_특징이 Private room in townhouse인 숙소의 비율 : 1.65%

숙소_특징이 Private room in condo인 숙소의 개수: 2178개
숙소_특징이 Private room in condo인 숙소의 비율 : 5.06%

숙소_특징이 Entire condo인 숙소의 개수: 5465개
숙소_특징이 Entire condo인 숙소의 비율 : 12.69%

숙소_특징이 Room in aparthotel인 숙소의 개수: 71개
숙소_특징이 Room in aparthotel인 숙소의 비율 : 0.16%

숙소_특징이 Entire serviced apartment인 숙소의 개수: 1267개
숙소_특징이 Entire serviced apartment인 숙소의 비율 : 2.94%

숙소_특징이 Private room in loft인 숙소의 개수

In [36]:
# 개수 내림차순으로 확인 

london_lists['숙소_특징'].value_counts()

Entire rental unit                    16943
Entire condo                           5465
Private room in home                   5273
Private room in rental unit            4845
Entire home                            3617
Private room in condo                  2178
Entire serviced apartment              1267
Private room in townhouse               711
Entire townhouse                        611
Private room in bed and breakfast       290
Room in hotel                           213
Entire loft                             187
Private room in guesthouse              179
Entire guesthouse                       146
Room in boutique hotel                  146
Private room in guest suite             126
Entire guest suite                      117
Private room in loft                     73
Room in aparthotel                       71
Private room in serviced apartment       68
Private room in casa particular          67
Entire vacation home                     62
Entire place                    

#### 숙소 유형별 숙소 특징값 개수 확인 

In [37]:
# 숙소_유형 == Private Room의 숙소_특징별 Count

london_lists.groupby(['숙소_유형', '숙소_특징']).size().reset_index(name='count').sort_values(by=['숙소_유형', 'count'], ascending=False, ignore_index=True)[:28]

,숙소_유형,숙소_특징,count
0,Private room,Private room in home,5273
1,Private room,Private room in rental unit,4845
2,Private room,Private room in condo,2178
3,Private room,Private room in townhouse,711
4,Private room,Private room in bed and breakfast,290
5,Private room,Room in hotel,213
6,Private room,Private room in guesthouse,179
7,Private room,Room in boutique hotel,146
8,Private room,Private room in guest suite,126
9,Private room,Private room in loft,73


In [38]:
# 숙소_유형 == Entire home/apt의 숙소_특징별 Count
london_lists.groupby(['숙소_유형', '숙소_특징']).size().reset_index(name='count').sort_values(by=['숙소_유형', 'count'], ascending=False, ignore_index=True)[28:]

,숙소_유형,숙소_특징,count
28,Entire home/apt,Entire rental unit,16943
29,Entire home/apt,Entire condo,5465
30,Entire home/apt,Entire home,3617
31,Entire home/apt,Entire serviced apartment,1267
32,Entire home/apt,Entire townhouse,611
33,Entire home/apt,Entire loft,187
34,Entire home/apt,Entire guesthouse,146
35,Entire home/apt,Entire guest suite,117
36,Entire home/apt,Entire vacation home,62
37,Entire home/apt,Entire place,42


상위 10개씩만 가져가보자. 

In [39]:
# 변수 정의 
sorted_PrivateRoom = london_lists.groupby(['숙소_유형', '숙소_특징']).size().reset_index(name='count').sort_values(by=['숙소_유형', 'count'], ascending=False, ignore_index=True)[:28].reset_index(drop=True)
sorted_EntireHomeApt = london_lists.groupby(['숙소_유형', '숙소_특징']).size().reset_index(name='count').sort_values(by=['숙소_유형', 'count'], ascending=False, ignore_index=True)[28:].reset_index(drop=True)

# 상위 10개 항목 추출
sorted_PrivateRoom_top10 = sorted_PrivateRoom['숙소_특징'][:10].values
sorted_EntireHomeApt_top10 = sorted_EntireHomeApt['숙소_특징'][:10].values

# 조건 생성
condition_PrivateRoom_top10 = london_lists['숙소_특징'].isin(sorted_PrivateRoom_top10)
condition_EntireHomeApt_top10 = london_lists['숙소_특징'].isin(sorted_EntireHomeApt_top10)

# 데이터 개수 확인
print(f'Private Room에서 가장 많은 숙소_특징 상위 10개의 총 개수: {london_lists[condition_PrivateRoom_top10].shape[0]}개')
print(f'Entire Home/Apt에서 가장 많은 숙소_특징 상위 10개의 총 개수: {london_lists[condition_EntireHomeApt_top10].shape[0]}개')
print()

# 제거할 데이터의 개수 및 비율 구하기 
print(f'숙소 특징에서 제거할 데이터의 개수: {london_lists[~(condition_PrivateRoom_top10 | condition_EntireHomeApt_top10)].shape[0]}개')
print(f'전체 데이터 대비 숙소 특징에서 제거할 데이터의 비율: {round((london_lists[~(condition_PrivateRoom_top10 | condition_EntireHomeApt_top10)].shape[0] / london_lists.shape[0]) * 100, 2)}%')

# 데이터 제거 
london_lists = london_lists[(condition_PrivateRoom_top10 | condition_EntireHomeApt_top10)]
print(f'제거 후 데이터 개수 : {london_lists.shape[0]}개')

Private Room에서 가장 많은 숙소_특징 상위 10개의 총 개수: 14034개
Entire Home/Apt에서 가장 많은 숙소_특징 상위 10개의 총 개수: 28457개

숙소 특징에서 제거할 데이터의 개수: 570개
전체 데이터 대비 숙소 특징에서 제거할 데이터의 비율: 1.32%
제거 후 데이터 개수 : 42491개


## 수용_인원수 
수용 가능 인원수도 숙소의 크기를 가늠할 수 있는 하나의 기준이 될 수 있지 않을까? 

In [40]:
london_lists['수용_인원수'].describe()

count    42491.000000
mean         3.361229
std          2.056741
min          1.000000
25%          2.000000
50%          3.000000
75%          4.000000
max         16.000000
Name: 수용_인원수, dtype: float64

In [41]:
# 수용_인원수 박스플랏 확인

import plotly.express as px

# 박스 플롯 생성
fig = px.box(london_lists, y=london_lists['수용_인원수'], title='Box Plot of 수용 인원수')

# 크기 설정 (너비: 6인치, 높이: 12인치)
fig.update_layout(
    width=6*96,  # 1인치 = 96픽셀
    height=12*96
)

# 출력
fig.show()

## 욕실수 / 침대수 / 침실수 
* 숙소_유형에 따라 욕실수, 침대수, 침실수의 갯수가 일정하게 다른 패턴이 있음 

<br />

* Entire home/apt는 우리가 상식적으로 생각할 수 있는 형태 
    * 단, Entire home의 경우 하나의 숙소가 아닌 여러 숙소의 값을 하나의 숙소 ID에 놔둔 경우가 있어서 제거 
    * 또한 Entire home의 경우 간혹 '침대수' 이상치를 발견 (>> 이건 실제로 침대가 많은 경우)

<br />

* Private Room의 경우 : 에어비앤비의 유니크 케이스 >> 집 전체에서 하나의 방을 빌려주는 경우 
    * 침실 수가 1이 아닌 경우엔 >> 숙소 전체의 방 개수를 지칭 
    * 욕실 수 역시 전체 욕실 수를 지칭

욕실수, 침대수, 침실수를 보는 이유: 숙소의 크기를 가늠할 수 있는 기준으로 활용할 수 있지 않나? 라는 가설 때문 
* 다시 말해 숙소의 크기가 클수록 가격이 비싸진다라는 가설을 증명하고 싶어서

<br />

* 그러면 '욕실수, 침대수, 침실수'가 숙소의 크기와 직결할 수 있는 '숙소_유형'에서만 사용하면 되지 않을까? 
    * Entire home에서는 '욕실수와 침실수'가 숙소의 크기와 관련이 있어보임 (침대수는 이상치 때문에 굳이 보지 않아도 될 것 같음)
    * 또한 Private Room의 경우엔 방 하나를 빌리는 형태이기 때문에 Room 개수를 1개로 통일시켜야 함으로 사실 상 컬럼값이 무의미
        * (욕실을 넣을 거면 전용 욕실이냐 공용욕실이냐로 나눠야 하는데 이걸 모든 데이터에 적용하는 것은 불가능)
        * 차라리 수용 인원 수를 비교해보는게 숙소의 크기를 가늠할 수 있는 기준이 될 수도 있을 듯

<br />

이런 식으로 숙소 유형에 따라 집어넣을 컬럼을 다르게 해서 접근해보자.

### Entire home/apt >> 수용인원, 침실수

In [42]:
# 'Entire home/apt' 실제 숙소 확인

# 조건 설정 : 욕실 수 6개 이상
condition_bathroom_outlier = london_url['욕실수'] >= 6
condition_room_entirehomeapt = london_url['숙소_유형'] == 'Entire home/apt'

# 조회할 컬럼 >> 결측치를 제거하면서 제거된 컬럼인지 확인도 해야함 
col_url = ['숙소_id', '숙소_url', '호스트_id', '슈퍼호스트', '숙소_유형', '수용_인원수','욕실수', '침실수', '침대수', '숙소_가격','리뷰수', '리뷰점수', '숙소_정확성_리뷰점수', '숙소_청결도_리뷰점수', '숙소_체크인_리뷰점수', '숙소_소통_리뷰점수', '숙소_위치_리뷰점수', '숙소_가격_리뷰점수']

# 데이터 조회
london_url[col_url][condition_bathroom_outlier & condition_room_entirehomeapt].sort_values(by=['욕실수', '침실수', '침대수'], ascending=False)[:20]

,숙소_id,숙소_url,호스트_id,슈퍼호스트,숙소_유형,수용_인원수,욕실수,침실수,침대수,숙소_가격,리뷰수,리뷰점수,숙소_정확성_리뷰점수,숙소_청결도_리뷰점수,숙소_체크인_리뷰점수,숙소_소통_리뷰점수,숙소_위치_리뷰점수,숙소_가격_리뷰점수
34670,39840632,https://www.airbnb.com/rooms/39840632,162523434,t,Entire home/apt,16,13.0,16.0,38.0,$600.00,1,5.00,5.00,5.00,5.00,3.00,4.00,5.00
9965,12598766,https://www.airbnb.com/rooms/12598766,36262172,t,Entire home/apt,16,10.0,22.0,16.0,"$1,247.00",40,4.83,4.78,4.88,4.85,4.98,4.50,4.60
49979,664992269841262049,https://www.airbnb.com/rooms/664992269841262049,108223343,f,Entire home/apt,10,9.0,9.0,10.0,$464.00,8,4.88,4.63,4.88,4.88,4.75,4.50,4.38
41630,50716053,https://www.airbnb.com/rooms/50716053,407980308,f,Entire home/apt,16,8.5,7.0,8.0,"$1,453.00",6,4.67,4.33,4.33,4.50,4.33,4.33,3.83
44069,53426653,https://www.airbnb.com/rooms/53426653,108223343,f,Entire home/apt,8,8.5,7.0,8.0,$300.00,11,4.73,4.64,4.82,4.73,4.45,4.27,4.27
59574,830516666111642155,https://www.airbnb.com/rooms/830516666111642155,536274836,f,Entire home/apt,16,8.0,10.0,10.0,"$1,275.00",1,5.00,5.00,5.00,5.00,5.00,5.00,5.00
22117,23757469,https://www.airbnb.com/rooms/23757469,67803292,f,Entire home/apt,16,7.5,7.0,10.0,"$1,100.00",20,4.80,4.90,4.70,5.00,5.00,4.95,4.95
75063,990354553225078956,https://www.airbnb.com/rooms/990354553225078956,297765625,f,Entire home/apt,14,7.0,7.0,7.0,"$1,000.00",1,3.00,4.00,5.00,4.00,3.00,4.00,4.00
39893,48281023,https://www.airbnb.com/rooms/48281023,121395448,f,Entire home/apt,10,7.0,6.0,8.0,"$2,500.00",3,5.00,4.33,4.67,5.00,4.67,5.00,4.33
77722,1013719570483982799,https://www.airbnb.com/rooms/1013719570483982799,544113276,f,Entire home/apt,12,7.0,6.0,8.0,$404.00,3,5.00,5.00,5.00,5.00,5.00,4.67,4.67


* 우선 1번 2번 숙소의 경우 (39840632 / 12598766)의 경우 하나의 숙소 안에 있는 욕실과 침실이 아니라서 제거 == 이상치 제거 

In [43]:
# 'Entire home/apt' 실제 숙소 확인

# 조건 설정 
# condition_bathroom_outlier = london_url['욕실수'] >= 6
condition_room_entirehomeapt = london_url['숙소_유형'] == 'Entire home/apt'

# 조회할 컬럼 >> 결측치를 제거하면서 제거된 컬럼인지 확인도 해야함 
col_url = ['숙소_id', '숙소_url', '호스트_id', '슈퍼호스트', '숙소_유형', '수용_인원수', '침실수', '숙소_가격','리뷰수', '리뷰점수', '숙소_정확성_리뷰점수', '숙소_청결도_리뷰점수', '숙소_체크인_리뷰점수', '숙소_소통_리뷰점수', '숙소_위치_리뷰점수', '숙소_가격_리뷰점수']

# 데이터 조회
london_url[col_url][condition_bathroom_outlier & condition_room_entirehomeapt].sort_values(by=['수용_인원수', '침실수'], ascending=False)[:20]

,숙소_id,숙소_url,호스트_id,슈퍼호스트,숙소_유형,수용_인원수,침실수,숙소_가격,리뷰수,리뷰점수,숙소_정확성_리뷰점수,숙소_청결도_리뷰점수,숙소_체크인_리뷰점수,숙소_소통_리뷰점수,숙소_위치_리뷰점수,숙소_가격_리뷰점수
9965,12598766,https://www.airbnb.com/rooms/12598766,36262172,t,Entire home/apt,16,22.0,"$1,247.00",40,4.83,4.78,4.88,4.85,4.98,4.50,4.60
34670,39840632,https://www.airbnb.com/rooms/39840632,162523434,t,Entire home/apt,16,16.0,$600.00,1,5.00,5.00,5.00,5.00,3.00,4.00,5.00
59574,830516666111642155,https://www.airbnb.com/rooms/830516666111642155,536274836,f,Entire home/apt,16,10.0,"$1,275.00",1,5.00,5.00,5.00,5.00,5.00,5.00,5.00
11498,13905660,https://www.airbnb.com/rooms/13905660,33549080,f,Entire home/apt,16,8.0,$894.00,91,4.51,4.44,4.42,4.77,4.78,5.00,4.49
28761,32278025,https://www.airbnb.com/rooms/32278025,33549080,f,Entire home/apt,16,8.0,"$1,374.00",2,5.00,4.50,5.00,4.50,5.00,5.00,4.00
22117,23757469,https://www.airbnb.com/rooms/23757469,67803292,f,Entire home/apt,16,7.0,"$1,100.00",20,4.80,4.90,4.70,5.00,5.00,4.95,4.95
41630,50716053,https://www.airbnb.com/rooms/50716053,407980308,f,Entire home/apt,16,7.0,"$1,453.00",6,4.67,4.33,4.33,4.50,4.33,4.33,3.83
32949,37770030,https://www.airbnb.com/rooms/37770030,238903426,t,Entire home/apt,16,6.0,$536.00,51,4.84,4.82,4.90,4.92,4.94,4.98,4.69
55022,758526294027255001,https://www.airbnb.com/rooms/758526294027255001,37056489,f,Entire home/apt,14,8.0,"$3,189.00",5,5.00,5.00,5.00,5.00,5.00,5.00,5.00
40523,49361737,https://www.airbnb.com/rooms/49361737,22164664,f,Entire home/apt,14,7.0,"$1,250.00",66,4.71,4.71,4.50,4.91,4.83,4.88,4.59


In [44]:
# 조회할 컬럼 지정 
col_lists = ['숙소_id', '호스트_id', '슈퍼호스트', '숙소_유형', '욕실수', '침실수', '침대수', '숙소_가격','리뷰수', '리뷰점수']

# 데이터 값 조회를 위한 조건 지정
id_39840632 = london_lists['숙소_id'] == 39840632
id_12598766 = london_lists['숙소_id'] == 12598766

# 데이터 조회 
london_lists[col_lists][id_39840632 | id_12598766]

# 데이터 제거(Outlier 제거)
london_lists = london_lists[~(id_39840632 | id_12598766)]

In [45]:
# 'Entire home/apt' 욕실수 / 침대수 기초 통계량 요약

# 조건 설정 : 욕실 수 6개 이상
# condition_bathroom_outlier = london_url['욕실수'] >= 6
condition_room_entirehomeapt = london_lists['숙소_유형'] == 'Entire home/apt'

london_lists[['수용_인원수', '욕실수', '침실수']][condition_room_entirehomeapt].describe()

,수용_인원수,욕실수,침실수
count,28455.000000,28455.000000,28455.000000
mean,4.088596,1.406554,1.713794
std,2.072198,0.665593,1.027066
min,1.000000,0.000000,0.000000
25%,2.000000,1.000000,1.000000
50%,4.000000,1.000000,2.000000
75%,5.000000,2.000000,2.000000
max,16.000000,9.000000,10.000000


#### 수용_인원수 박스플랏 확인 및 이상치 확인

In [46]:
# 'Entire home/apt' 수용_인원수 박스플랏 확인

import plotly.express as px

# 박스 플롯 생성
fig = px.box(london_lists, y=london_lists['수용_인원수'][condition_room_entirehomeapt], title='Box Plot of 수용인원수 of Entire home/apt')

# 크기 설정 (너비: 6인치, 높이: 12인치)
fig.update_layout(
    width=6*96,  # 1인치 = 96픽셀
    height=12*96
)

# 출력
fig.show()

In [47]:
# IQR 정의
Q1 = london_lists['수용_인원수'][condition_room_entirehomeapt].quantile(.25)
Q2 = london_lists['수용_인원수'][condition_room_entirehomeapt].quantile(.50)
Q3 = london_lists['수용_인원수'][condition_room_entirehomeapt].quantile(.75)
IQR = Q3 - Q1 

# 출력
print(Q1)
print(Q2)
print(Q3)
print('IQR = ', IQR) 

# Outlier 조건 설정
condition_accommodates_upper_outliers = london_lists['수용_인원수'][condition_room_entirehomeapt] > (Q3 + IQR * 1.5)
# condition_accommodates_lower_outliers = london_lists['수용_인원수'][condition_room_entirehomeapt] < (Q1 - IQR * 1.5)

# Upper & Lower Outlier >> 변수로 정리
accommodates_upper_outliers = london_lists['수용_인원수'][condition_room_entirehomeapt][condition_accommodates_upper_outliers]
# accommodates_lower_outliers = london_lists['수용_인원수'][condition_room_entirehomeapt][condition_accommodates_lower_outliers]

# 출력
print(accommodates_upper_outliers.sort_values(ascending=False))
# print(accommodates_lower_outliers.sort_values(ascending=False))

2.0
4.0
5.0
IQR =  3.0
51468    16
28509    16
55026    16
57125    16
59574    16
         ..
57915    10
57429    10
27191    10
57256    10
90117    10
Name: 수용_인원수, Length: 603, dtype: int64


#### 욕실수 박스플랏 확인 및 이상치 확인 

In [48]:
# 'Entire home/apt' 욕실수 박스플랏 확인

import plotly.express as px

# 박스 플롯 생성
fig = px.box(london_lists, y=london_lists['욕실수'][condition_room_entirehomeapt], title='Box Plot of 욕실수 of Entire home/apt')

# 크기 설정 (너비: 6인치, 높이: 12인치)
fig.update_layout(
    width=6*96,  # 1인치 = 96픽셀
    height=12*96
)

# 출력
fig.show()

In [49]:
# IQR 정의
Q1 = london_lists['욕실수'][condition_room_entirehomeapt].quantile(.25)
Q2 = london_lists['욕실수'][condition_room_entirehomeapt].quantile(.50)
Q3 = london_lists['욕실수'][condition_room_entirehomeapt].quantile(.75)
IQR = Q3 - Q1 

# 출력
print('Q1 = ', Q1)
print('Q2 = ', Q2)
print('Q3 = ', Q3)
print('IQR = ', IQR) 

# Upper Outlier 조건 설정
condition_bathroom_upper_outliers = london_lists['욕실수'][condition_room_entirehomeapt] > (Q3 + IQR * 1.5)

# Upper & Lower Outlier >> 변수로 정리
bathroom_upper_outliers = london_lists['욕실수'][condition_room_entirehomeapt][condition_bathroom_upper_outliers]

# 출력
bathroom_upper_outliers.sort_values(ascending=False)

Q1 =  1.0
Q2 =  1.0
Q3 =  2.0
IQR =  1.0


49979    9.0
44069    8.5
59574    8.0
22117    7.5
77722    7.0
        ... 
44620    4.0
44428    4.0
44219    4.0
42925    4.0
90117    4.0
Name: 욕실수, Length: 240, dtype: float64

#### 침실수 박스플랏 확인 및 이상치 확인 

In [50]:
# 'Entire home/apt' 침실수 박스플랏 확인

import plotly.express as px

# 박스 플롯 생성
fig = px.box(london_lists, y=london_lists['침실수'][condition_room_entirehomeapt], title='Box Plot of 침실수 of Entire home/apt')

# 크기 설정 (너비: 6인치, 높이: 12인치)
fig.update_layout(
    width=6*96,  # 1인치 = 96픽셀
    height=12*96
)

# 출력
fig.show()

In [51]:
# IQR 정의
Q1 = london_lists['침실수'][condition_room_entirehomeapt].quantile(.25)
Q2 = london_lists['침실수'][condition_room_entirehomeapt].quantile(.50)
Q3 = london_lists['욕실수'][condition_room_entirehomeapt].quantile(.75)
IQR = Q3 - Q1 

# 출력
print('Q1 = ', Q1)
print('Q2 = ', Q2)
print('Q3 = ',Q3)
print('IQR = ', IQR) 

# Upper Outlier 조건 설정
condition_bedroom_upper_outliers = london_lists['침실수'][condition_room_entirehomeapt] > (Q3 + IQR * 1.5)

# Upper & Lower Outlier >> 변수로 정리
bedroom_upper_outliers = london_lists['침실수'][condition_room_entirehomeapt][condition_bedroom_upper_outliers]

# 출력
bedroom_upper_outliers.sort_values(ascending=False)

Q1 =  1.0
Q2 =  2.0
Q3 =  2.0
IQR =  1.0


59574    10.0
49979     9.0
80790     8.0
24215     8.0
36229     8.0
         ... 
49646     4.0
49573     4.0
16720     4.0
49517     4.0
24        4.0
Name: 침실수, Length: 1665, dtype: float64

### Private Room >> 수용인원, 침대수


Private Room의 경우 
* 침실 수가 1이 아닌 경우엔 >> 숙소 전체의 방 개수를 지칭 
* 욕실 수 역시 전체 욕실 수를 지칭

In [52]:
# 'Private room' 실제 숙소 확인

# 조건 설정 
condition_room_privateroom = london_url['숙소_유형'] == 'Private room'

# 조회할 컬럼 >> 결측치를 제거하면서 제거된 컬럼인지 확인도 해야함 
col_url = ['숙소_id', '숙소_url', '호스트_id', '슈퍼호스트', '숙소_유형', '수용_인원수', '침대수', '숙소_가격','리뷰수', '리뷰점수', '숙소_정확성_리뷰점수', '숙소_청결도_리뷰점수', '숙소_체크인_리뷰점수', '숙소_소통_리뷰점수', '숙소_위치_리뷰점수', '숙소_가격_리뷰점수']

# 데이터 조회
london_url[col_url][condition_room_privateroom].sort_values(by=['수용_인원수', '침대수'], ascending=False)[:50]

,숙소_id,숙소_url,호스트_id,슈퍼호스트,숙소_유형,수용_인원수,침대수,숙소_가격,리뷰수,리뷰점수,숙소_정확성_리뷰점수,숙소_청결도_리뷰점수,숙소_체크인_리뷰점수,숙소_소통_리뷰점수,숙소_위치_리뷰점수,숙소_가격_리뷰점수
28628,32205440,https://www.airbnb.com/rooms/32205440,241641109,f,Private room,16,10.0,"$2,999.00",6,4.33,4.33,3.83,5.00,4.83,5.00,4.33
4780,6742681,https://www.airbnb.com/rooms/6742681,33802425,f,Private room,16,8.0,$924.00,13,4.85,4.77,4.69,4.92,5.00,4.50,4.75
46058,587051580777165921,https://www.airbnb.com/rooms/587051580777165921,38851959,t,Private room,13,13.0,$25.00,8,5.00,5.00,5.00,5.00,5.00,5.00,5.00
15094,17925370,https://www.airbnb.com/rooms/17925370,122908394,t,Private room,13,2.0,$300.00,3,4.67,4.67,4.67,4.67,5.00,4.67,5.00
46216,585650923890349995,https://www.airbnb.com/rooms/585650923890349995,38851959,t,Private room,13,NaN,NaN,7,5.00,5.00,5.00,5.00,5.00,5.00,5.00
48444,641658136008421995,https://www.airbnb.com/rooms/641658136008421995,1944259,f,Private room,12,10.0,$260.00,44,4.75,4.73,4.77,4.95,4.82,4.61,4.64
79021,1023737653404459810,https://www.airbnb.com/rooms/1023737653404459810,129141144,f,Private room,12,9.0,$546.00,21,4.81,4.90,4.67,4.95,5.00,4.81,4.76
20906,22606388,https://www.airbnb.com/rooms/22606388,126820422,f,Private room,12,6.0,$200.00,39,4.72,4.67,4.72,4.97,4.95,4.69,4.72
15033,17783932,https://www.airbnb.com/rooms/17783932,11060461,f,Private room,12,NaN,NaN,2,5.00,5.00,5.00,5.00,5.00,5.00,5.00
51926,702105394515370244,https://www.airbnb.com/rooms/702105394515370244,454080117,f,Private room,10,9.0,$60.00,5,4.20,4.40,3.40,4.40,4.40,4.80,4.20


In [53]:
# 'Private Room' 수용인원수 / 침대수 기초 통계량 요약

# 조건 설정 : 욕실 수 6개 이상
# condition_bathroom_outlier = london_url['욕실수'] >= 6
condition_room_privateroom = london_lists['숙소_유형'] == 'Private room'

london_lists[['수용_인원수', '침대수']][condition_room_privateroom].describe()

,수용_인원수,침대수
count,14034.000000,14034.000000
mean,1.884637,1.142582
std,0.908648,0.656151
min,1.000000,0.000000
25%,1.000000,1.000000
50%,2.000000,1.000000
75%,2.000000,1.000000
max,16.000000,14.000000


#### Private room 수용인원수 박스플랏 및 이상치 확인 

In [54]:
# 'Private room' 수용_인원수 박스플랏 확인

import plotly.express as px

# 박스 플롯 생성
fig = px.box(london_lists, y=london_lists['수용_인원수'][condition_room_privateroom], title='Box Plot of 수용인원수 of Private room')

# 크기 설정 (너비: 6인치, 높이: 12인치)
fig.update_layout(
    width=6*96,  # 1인치 = 96픽셀
    height=12*96
)

# 출력
fig.show()

In [55]:
# IQR 정의
Q1 = london_lists['수용_인원수'][condition_room_privateroom].quantile(.25)
Q2 = london_lists['수용_인원수'][condition_room_privateroom].quantile(.50)
Q3 = london_lists['수용_인원수'][condition_room_privateroom].quantile(.75)
IQR = Q3 - Q1 

# 출력
print(Q1)
print(Q2)
print(Q3)
print('IQR = ', IQR) 

# Upper Outlier 조건 설정
condition_accommodates_upper_outliers = london_lists['수용_인원수'][condition_room_privateroom] > (Q3 + IQR * 1.5)

# Upper & Lower Outlier >> 변수로 정리
accommodates_upper_outliers = london_lists['수용_인원수'][condition_room_privateroom][condition_accommodates_upper_outliers]

# 출력
accommodates_upper_outliers.sort_values(ascending=False)

1.0
2.0
2.0
IQR =  1.0


4780     16
28628    16
15094    13
46058    13
48444    12
         ..
40081     4
40129     4
40190     4
40318     4
90568     4
Name: 수용_인원수, Length: 787, dtype: int64

## 편의시설 
* 결론 : 가격과의 관계 에측에서는 편의시설이 유의미한 인사이트를 줄것 같지 않다고 판단하여 컬럼 제거 

In [56]:
london_lists['편의시설'].unique()

array(['["Toaster", "Room-darkening shades", "Coffee maker: french press", "Wifi", "AC - split type ductless system", "Dishwasher", "Cleaning products", "Luggage dropoff allowed", "Dishes and silverware", "Hot water kettle", "Free dryer \\u2013 In unit", "Extra pillows and blankets", "Free washer \\u2013 In unit", "Hot water", "Kitchen", "Laundromat nearby", "Bathtub", "Clothing storage", "Wine glasses", "Central heating", "Bed linens", "Single level home", "Portable fans", "Microwave", "Private patio or balcony", "Essentials", "Freezer", "Iron", "Bosch stainless steel gas stove", "Refrigerator", "Hangers", "Cleaning available during stay", "Carbon monoxide alarm", "Coffee", "55\\" HDTV with Netflix, premium cable, standard cable", "Smoke alarm", "Long term stays allowed", "Dedicated workspace", "Cooking basics", "Paid parking off premises", "Hair dryer", "Dining table", "Ethernet connection", "Drying rack for clothing", "Oven", "Books and reading material"]',
       '["Toaster", "Room

In [57]:
london_lists['편의시설'].nunique()

40559

가격과의 비교에서 편의시설을 통해서 유의미한 관계를 증명하기는 어려울 것으로 판단
* 추후 별도의 컬럼으로 분리해서 각 항목 별로 존재하는지 없는지 분류해야할 듯 

In [58]:
# 편의시설 컬럼도 우선 제거 

london_lists.drop(columns='편의시설', inplace=True)

## 숙소_유형

In [59]:
london_lists['숙소_유형'].unique()

array(['Entire home/apt', 'Private room'], dtype=object)

In [60]:
london_lists['숙소_유형'].value_counts()

Entire home/apt    28455
Private room       14034
Name: 숙소_유형, dtype: int64

In [61]:
# 전체 데이터에서 각 숙소 유형별 개수 및 전체 대비 비율 계산하기

CategoricalData_Count_Ratio(london_lists, '숙소_유형')

전체 숙소 개수: 42489개

숙소_유형이 Entire home/apt인 숙소의 개수: 28455개
숙소_유형이 Entire home/apt인 숙소의 비율 : 66.97%

숙소_유형이 Private room인 숙소의 개수: 14034개
숙소_유형이 Private room인 숙소의 비율 : 33.03%



## 숙소_지역

In [62]:
print(london_lists['숙소_지역'].unique())
print(london_lists['숙소_지역'].nunique())

['Hackney' 'Islington' 'Kensington and Chelsea' 'Lambeth' 'Westminster'
 'Richmond upon Thames' 'Tower Hamlets' 'Wandsworth' 'Camden' 'Haringey'
 'Enfield' 'Hammersmith and Fulham' 'Southwark' 'Brent' 'Waltham Forest'
 'Barnet' 'Barking and Dagenham' 'Croydon' 'Ealing' 'Lewisham' 'Merton'
 'City of London' 'Greenwich' 'Hounslow' 'Newham' 'Hillingdon' 'Harrow'
 'Redbridge' 'Kingston upon Thames' 'Bromley' 'Sutton' 'Havering' 'Bexley']
33


In [63]:
london_lists['숙소_지역'].value_counts()

Westminster               5309
Kensington and Chelsea    3112
Tower Hamlets             3068
Camden                    2967
Hackney                   2522
Southwark                 2200
Lambeth                   2143
Wandsworth                2061
Islington                 1956
Hammersmith and Fulham    1917
Brent                     1400
Newham                    1134
Ealing                    1128
Barnet                    1118
Lewisham                  1116
Haringey                  1048
Greenwich                  968
Waltham Forest             836
Croydon                    786
Merton                     678
Richmond upon Thames       654
Hounslow                   645
Hillingdon                 469
Enfield                    454
Redbridge                  418
Bromley                    414
Kingston upon Thames       344
Barking and Dagenham       317
Harrow                     302
Bexley                     288
City of London             280
Havering                   236
Sutton  

In [64]:
# 각 숙소_지역별 개수 및 비율 확인 

CategoricalData_Count_Ratio(london_lists, '숙소_지역')

전체 숙소 개수: 42489개

숙소_지역이 Hackney인 숙소의 개수: 2522개
숙소_지역이 Hackney인 숙소의 비율 : 5.94%

숙소_지역이 Islington인 숙소의 개수: 1956개
숙소_지역이 Islington인 숙소의 비율 : 4.6%

숙소_지역이 Kensington and Chelsea인 숙소의 개수: 3112개
숙소_지역이 Kensington and Chelsea인 숙소의 비율 : 7.32%

숙소_지역이 Lambeth인 숙소의 개수: 2143개
숙소_지역이 Lambeth인 숙소의 비율 : 5.04%

숙소_지역이 Westminster인 숙소의 개수: 5309개
숙소_지역이 Westminster인 숙소의 비율 : 12.49%

숙소_지역이 Richmond upon Thames인 숙소의 개수: 654개
숙소_지역이 Richmond upon Thames인 숙소의 비율 : 1.54%

숙소_지역이 Tower Hamlets인 숙소의 개수: 3068개
숙소_지역이 Tower Hamlets인 숙소의 비율 : 7.22%

숙소_지역이 Wandsworth인 숙소의 개수: 2061개
숙소_지역이 Wandsworth인 숙소의 비율 : 4.85%

숙소_지역이 Camden인 숙소의 개수: 2967개
숙소_지역이 Camden인 숙소의 비율 : 6.98%

숙소_지역이 Haringey인 숙소의 개수: 1048개
숙소_지역이 Haringey인 숙소의 비율 : 2.47%

숙소_지역이 Enfield인 숙소의 개수: 454개
숙소_지역이 Enfield인 숙소의 비율 : 1.07%

숙소_지역이 Hammersmith and Fulham인 숙소의 개수: 1917개
숙소_지역이 Hammersmith and Fulham인 숙소의 비율 : 4.51%

숙소_지역이 Southwark인 숙소의 개수: 2200개
숙소_지역이 Southwark인 숙소의 비율 : 5.18%

숙소_지역이 Brent인 숙소의 개수: 1400개
숙소_지역이 Brent인 숙소의 비율 : 3.29%



### 런던 자치구 목록 

시티오브런던 (런던 자치구가 아님), 시티오브웨스트민스터, 켄징턴 첼시, 해머스미스 풀럼, 원즈워스, 램버스, 서더크, 타워햄리츠, 해크니, 이즐링턴, 캠던, 브렌트, 일링 ,하운즐로 ,리치먼드어폰템스, 킹스턴어폰템스 ,머턴, 서턴, 크로이던, 브롬리, 루이셤, 그리니치, 벡슬리, 헤이버링, \
바킹 대거넘, 레드브리지, 뉴엄, 월섬포리스트, 해링게이, 엔필드, 바닛, 해로, 힐링던

<h4>숙소지역 특징 by 예진님</h4>

1. Hackney(해크니): <span style="color:yellow">동런던</span>에 위치한 트렌디한 지역 / <span style="color:yellow">예술가와 창의적</span>인 커뮤니티가 많이 거주

2. Islington(이슬링턴): <span style="color:yellow">중북부</span>에 위치 / <span style="color:yellow">고급 주거 지역</span> (고풍스러운 타운하우스와 다양한 음식점, 극장)

3. Kensington and Chelsea(켄싱턴 앤드 첼시): <span style="color:yellow">서쪽</span>에 위치 / <span style="color:yellow">가장 부유한 지역</span> (고급 부티크와 박물관, 아름다운 주택)

4. Lambeth(램버스): <span style="color:yellow">남쪽</span>위치한 지역 / 런던 아이와 사우스뱅크 센터 같은 <span style="color:yellow">주요 관광지</span> (다양하고 활기찬 커뮤니티)

5. Brent(브렌트): <span style="color:yellow">북서쪽</span>위치 / <span style="color:yellow">스포츠와 음악 이벤트</span> (웸블리 스타디움이 있는 곳)

6. Southwark(사우스워크): <span style="color:yellow">남동쪽</span> 위치 /   <span style="color:yellow">문화 관광 거리</span> (테이트 모던 미술관과 버러 마켓이 유명)

7. Westminster(웨스트민스터): <span style="color:yellow">중심부</span>위치 / <span style="color:yellow">정부 기관 지역</span> (국회의사당, 버킹엄 궁전, 트라팔가 광장)

8. Wandsworth(원스워스):<span style="color:yellow">남서쪽</span>위치 / 
<span style="color:yellow">주거 지역</span> (큰 공원과 가족 친화적인 분위기가 특징)

9. Tower Hamlets(타워햄리츠): <span style="color:yellow">동쪽</span> 위치 / 런던 타워와 카나리 워프 같은 <span style="color:yellow">역사적이고 상업적인 중심지</span>

10. Richmond upon Thames(리치먼드): <span style="color:yellow">남서쪽</span>에 위치 / <span style="color:yellow">풍요롭고 피크닉 장소</span> (리치몬드 공원과 템스 강이 유명)

11. Newham(뉴험): <span style="color:yellow">동쪽</span>에 위치 / <span style="color:yellow">다문화 지역</span> (올림픽 공원과 웨스트필드 쇼핑센터)

12. Camden(캠던): <span style="color:yellow">북부</span>에 위치 / <span style="color:yellow">보헤미안 지역</span> (캠든 마켓과 음악 공연장이 많음)

13. Haringey(하린지): <span style="color:yellow">북쪽</span>에 위치 / <span style="color:yellow">주거 지역</span> (알렉산드라 팰리스와 큰 공원)

14. Enfield(엔필드): <span style="color:yellow">북쪽 외곽</span>에 위치 / <span style="color:yellow">자연적인 지역</span> (넓은 녹지와 가족 친화적인 분위기가 특징)

15. Hammersmith and Fulham(해머스미스 앤드 풀럼): <span style="color:yellow">서쪽</span>에 위치 / <span style="color:yellow">국제 기업의 사무소 지역</span> (템스 강변의 아름다운 경치와 고급 주택들)

16. Waltham Forest(월섬 포레스트): <span style="color:yellow">북동쪽</span>에 위치 / <span style="color:yellow">녹지와 공원 지역</span> (예술과 문화적 활동이 활발한 곳)

17. Barnet(바넷): <span style="color:yellow">북서쪽</span>에 위치 / <span style="color:yellow">큰 주거 지역</span> (가족과 함께 살기 좋은 환경을 제공)

18. Hounslow(하운슬로): <span style="color:yellow">서쪽 외곽</span>에 위치 / <span style="color:yellow">히드로 공항과 가까운</span>특징 (교통이 편리)

19. Barking and Dagenham(바킹 앤드 다게넘): <span style="color:yellow">동쪽 외곽</span>에 위치 / <span style="color:yellow">공업 지대와 주거 지역이 혼재</span> 다문화 지역

20. Croydon(크로이던): <span style="color:yellow">남쪽</span>에 위치 / <span style="color:yellow">큰 상업 중심지</span> (쇼핑몰과 비즈니스 허브가 많음)

21. Lewisham(루이섬): <span style="color:yellow">남동쪽</span>에 위치 / <span style="color:yellow">다양한 문화적 배경을 가진 사람들이 거주</span> (활기찬 시장이 있다)

22. Ealing(일링): <span style="color:yellow">서쪽</span>에 위치 / <span style="color:yellow">가족 친화적인 분위기</span> (철도역이 있고 큰 공원)

23. Merton(머튼): <span style="color:yellow">남서쪽</span>에 위치 / <span style="color:yellow">윔블던 테니스 챔피언십이 열리는 윔블던이 있음</span> 

24. City of London: <span style="color:yellow">북동쪽</span>에 위치 / <span style="color:yellow">런던의 금융 중심지</span> (고층 건물과 역사적인 랜드마크가 혼재)

25. Greenwich(그리니치): <span style="color:yellow">남동쪽</span>에 위치 / <span style="color:yellow">그리니치 천문대 유명</span> (해양 박물관이 유명, 관광지)

26. Hillingdon(힐링던): <span style="color:yellow">서쪽 외곽</span>에 위치 / <span style="color:yellow">히드로 공항이 있다.</span>

27. Bromley(브롬리): <span style="color:yellow">남동쪽 외곽</span>에 위치 / <span style="color:yellow"> 큰 주거 지역</span> (녹지가 많고 가족들이 많이 거주)

28. Havering(해버링): <span style="color:yellow">동쪽 외곽</span>에 위치 / <span style="color:yellow">주거 지역 느낌</span> 평온하고 넓은 녹지가 있다.

29. Harrow(해로): <span style="color:yellow">북서쪽</span>에 위치 / <span style="color:yellow">주거 지역</span> (좋은 교육 시설이 많음)

30. Bexley(벡슬리): <span style="color:yellow">남동쪽 외곽</span>에 위치 / <span style="color:yellow">주거 지역</span> 가족 친화적인 분위기와 공원이 많습니다.

31. Redbridge(레드브리지): <span style="color:yellow">북동쪽</span>에 위치 / <span style="color:yellow">주거 지역</span> (다양한 커뮤니티와 녹지가 있다)

32. Kingston upon Thames(킹스턴): <span style="color:yellow">남서쪽</span>에 위치 / <span style="color:yellow">쇼핑 지역이 유명</span> (템스 강변의 아름다운 경치)

33. Sutton(서턴): <span style="color:yellow">남쪽 외곽</span>에 위치 / <span style="color:yellow">주거 지역</span> (가족들이 살기 좋은 환경을 제공)

## 숙소_가격

* 기준 통화 : 달러
* 달러 표시 제거 후 타입 변경

In [65]:
# 데이터 타입 변경
london_lists['숙소_가격'] = london_lists['숙소_가격'].str.lstrip('$').str.replace(',', '').astype('float')
london_url['숙소_가격'] = london_url['숙소_가격'].str.lstrip('$').str.replace(',', '').astype('float')

# 변경된 내역 확인 
london_lists.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42489 entries, 0 to 90735
Data columns (total 25 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   숙소_id        42489 non-null  int64  
 1   숙소_url       42489 non-null  object 
 2   호스트_id       42489 non-null  int64  
 3   슈퍼호스트        42489 non-null  object 
 4   숙소_지역        42489 non-null  object 
 5   숙소_특징        42489 non-null  object 
 6   숙소_유형        42489 non-null  object 
 7   수용_인원수       42489 non-null  int64  
 8   욕실수          42489 non-null  float64
 9   침실수          42489 non-null  float64
 10  침대수          42489 non-null  float64
 11  숙소_가격        42489 non-null  float64
 12  예약_가능여부      42489 non-null  object 
 13  리뷰수          42489 non-null  int64  
 14  12개월_리뷰수     42489 non-null  int64  
 15  30일_리뷰수      42489 non-null  int64  
 16  마지막_리뷰       42489 non-null  object 
 17  리뷰점수         42489 non-null  float64
 18  숙소_정확성_리뷰점수  42489 non-null  float64
 19  숙소_청

In [66]:
london_lists['숙소_가격'].describe()

count    42489.000000
mean       165.598908
std        423.604661
min          0.000000
25%         71.000000
50%        121.000000
75%        195.000000
max      53588.000000
Name: 숙소_가격, dtype: float64

In [67]:
import plotly.express as px

# 박스 플롯 생성
fig = px.box(london_lists, y=london_lists['숙소_가격'], title='Box Plot of 숙소 가격')

# 크기 설정 
fig.update_layout(
    width=6*96,  # 1인치 = 96픽셀
    height=16*96
)

# 출력
fig.show()


### 실제 숙소 정보와 비교를 통한 데이터 확인

In [68]:
london_lists[col_lists].sort_values(by='숙소_가격', ascending=False)[:30]

,숙소_id,호스트_id,슈퍼호스트,숙소_유형,욕실수,침실수,침대수,숙소_가격,리뷰수,리뷰점수
10492,13254774,34349317,f,Private room,1.0,1.0,1.0,53588.0,97,4.85
27143,30004985,225239697,f,Entire home/apt,1.0,1.0,1.0,50000.0,5,5.00
7673,9721759,50179961,f,Entire home/apt,1.0,3.0,5.0,25000.0,8,4.63
80543,1036166747787748467,152456059,f,Private room,1.0,1.0,1.0,10000.0,1,4.00
852,698606,3581160,t,Private room,1.0,1.0,1.0,9999.0,359,4.96
7383,9470827,5484135,f,Private room,1.0,1.0,1.0,8000.0,8,4.75
34302,39383869,302514116,f,Entire home/apt,3.5,4.0,4.0,7000.0,5,4.80
33290,38157709,110810758,f,Private room,1.0,1.0,1.0,6000.0,83,4.77
38282,44732052,89693959,f,Private room,1.0,1.0,1.0,5749.0,173,4.53
38253,44696103,89693959,f,Private room,1.0,1.0,1.0,5748.0,69,4.57


In [69]:
london_url[col_url].sort_values(by='숙소_가격', ascending=False)[:50]

,숙소_id,숙소_url,호스트_id,슈퍼호스트,숙소_유형,수용_인원수,침대수,숙소_가격,리뷰수,리뷰점수,숙소_정확성_리뷰점수,숙소_청결도_리뷰점수,숙소_체크인_리뷰점수,숙소_소통_리뷰점수,숙소_위치_리뷰점수,숙소_가격_리뷰점수
10492,13254774,https://www.airbnb.com/rooms/13254774,34349317,f,Private room,1,1.0,53588.0,97,4.85,4.96,4.66,4.99,4.99,4.98,4.87
27143,30004985,https://www.airbnb.com/rooms/30004985,225239697,f,Entire home/apt,2,1.0,50000.0,5,5.00,4.80,4.60,4.60,5.00,5.00,4.80
7673,9721759,https://www.airbnb.com/rooms/9721759,50179961,f,Entire home/apt,6,5.0,25000.0,8,4.63,4.88,4.25,4.75,5.00,4.63,4.75
80543,1036166747787748467,https://www.airbnb.com/rooms/1036166747787748467,152456059,f,Private room,2,1.0,10000.0,1,4.00,5.00,5.00,4.00,5.00,3.00,4.00
852,698606,https://www.airbnb.com/rooms/698606,3581160,t,Private room,2,1.0,9999.0,359,4.96,4.97,4.98,4.99,4.99,4.88,4.94
7383,9470827,https://www.airbnb.com/rooms/9470827,5484135,f,Private room,1,1.0,8000.0,8,4.75,4.75,5.00,5.00,4.88,4.75,4.63
34302,39383869,https://www.airbnb.com/rooms/39383869,302514116,f,Entire home/apt,4,4.0,7000.0,5,4.80,5.00,4.60,4.60,5.00,5.00,4.80
33290,38157709,https://www.airbnb.com/rooms/38157709,110810758,f,Private room,2,1.0,6000.0,83,4.77,4.82,4.67,4.87,4.96,4.80,4.84
38282,44732052,https://www.airbnb.com/rooms/44732052,89693959,f,Private room,2,1.0,5749.0,173,4.53,4.68,4.54,4.84,4.78,4.89,4.63
38253,44696103,https://www.airbnb.com/rooms/44696103,89693959,f,Private room,1,1.0,5748.0,69,4.57,4.68,4.55,4.88,4.74,4.86,4.52


숙소 가격 말이 안됨. 
- 50,000불 = 5천만원 (1:1000으로만 계산을 해도) 
- 정말 최고의 하이엔드 호텔이라면 몰라도... 더군다나 Private Room이?? 

<br />

직접 확인 
1. 진짜 그 가격이 맞는데 아무도 이용을 안함 (리뷰가 있어도 다 몇 년 전 리뷰 > 여기서 중요한게 최신 리뷰)
2. 최근 리뷰가 있으면 가격이 수정되었음 
3. 영업 중지

1. 13254774 : 영업 중지 (https://www.airbnb.com/rooms/13254774)
2. 30004985 : 일박당 한화 1700만원, 최근 리뷰 없음 (https://www.airbnb.com/rooms/30004985)
3. 9721759 : 일박당 한화 4000만원, 최근 리뷰 없음 (https://www.airbnb.com/rooms/9721759)
4. 1036166747787748467: 일박당 20만원 >> 가격 정보 잘못됨 (https://www.airbnb.com/rooms/1036166747787748467)
5. 698606: 일박당 12만원 >> 가격 정보 잘못됨 (https://www.airbnb.com/rooms/698606)
6. 9470827: 일박당 1400만원 >> 최근 후기 없음 (https://www.airbnb.com/rooms/9470827)
7. 39383869	: 숙소 없음 (https://www.airbnb.com/rooms/39383869)
8. 38157709: 일박당 1000만원 >> 최근 리뷰 없음 (https://www.airbnb.com/rooms/38157709)
9. 44732052	: 일박당 12만원 >> 가격 정보 잘못됨 (https://www.airbnb.com/rooms/44732052)
10. 44696103 : 일박당 19만원 >> 가격 정보 잘못됨 (https://www.airbnb.com/rooms/44696103)


- 위에 뽑아 놓은 Top 30은 그냥 다 세가지 경우에 해당됨 
- 이거 어디까지 어떻게 지워야할까

가격 결측치 컬럼의 공통점 1 
* 최근 리뷰가 없다.
* 최근 리뷰가 있으면 가격이 수정되어 있다. 

## 최근 리뷰

In [70]:
london_lists[['12개월_리뷰수', '마지막_리뷰']]

,12개월_리뷰수,마지막_리뷰
0,26,2024-02-03
1,1,2023-11-26
2,2,2023-05-01
3,1,2023-04-29
4,25,2024-03-17
...,...,...
90451,1,2024-03-17
90568,1,2024-03-17
90590,2,2024-03-17
90648,1,2024-03-19


In [71]:
london_lists[['12개월_리뷰수', '마지막_리뷰']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42489 entries, 0 to 90735
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   12개월_리뷰수  42489 non-null  int64 
 1   마지막_리뷰    42489 non-null  object
dtypes: int64(1), object(1)
memory usage: 995.8+ KB


### 마지막 리뷰


In [72]:
# 데이터 타입 변경 >> datetime
london_lists['마지막_리뷰'] = pd.to_datetime(london_lists['마지막_리뷰'])


# 마지막 리뷰가 적힌 년도 >> 파생변수 생성
london_lists['마지막_리뷰_year'] = london_lists['마지막_리뷰'].dt.year

# 마지막 리뷰 년도별 숙소 가격 기초 통계 요약
마지막리뷰_년도별_숙소가격 = \
london_lists.groupby('마지막_리뷰_year')['숙소_가격'].agg(
    count='size',
    mean='mean',
    std='std',
    min='min',
    max='max',
    median='median'
).reset_index().sort_values(by='마지막_리뷰_year', ascending=False).reset_index(drop=True)

마지막리뷰_년도별_숙소가격

,마지막_리뷰_year,count,mean,std,min,max,median
0,2024,23088,147.132320,153.576038,8.0,10000.0,115.0
1,2023,15715,173.697741,196.832499,0.0,9999.0,130.0
2,2022,2043,221.011258,1133.870084,17.0,50000.0,135.0
3,2021,400,222.955000,462.607639,15.0,5074.0,125.0
4,2020,373,400.351206,2809.661093,1.0,53588.0,129.0
5,2019,451,281.026608,1220.768053,19.0,25000.0,120.0
6,2018,191,210.743455,399.397207,23.0,5100.0,125.0
7,2017,117,191.512821,235.129481,24.0,2000.0,125.0
8,2016,65,320.323077,1035.888947,26.0,8000.0,136.0
9,2015,30,163.866667,129.316434,40.0,513.0,114.5


In [73]:
# 마지막 리뷰 년도별 리뷰수 기초 통계 요약
마지막리뷰_년도별_리뷰수 = \
london_lists.groupby('마지막_리뷰_year')['리뷰수'].agg(
    count='size',
    mean='mean',
    std='std',
    min='min',
    max='max',
    median='median'
).reset_index().sort_values(by='마지막_리뷰_year', ascending=False).reset_index(drop=True)

마지막리뷰_년도별_리뷰수

,마지막_리뷰_year,count,mean,std,min,max,median
0,2024,23088,38.338184,66.371377,1,1457,15.0
1,2023,15715,18.376837,30.336160,1,497,8.0
2,2022,2043,12.643661,23.633719,1,337,5.0
3,2021,400,15.867500,33.472915,1,350,5.0
4,2020,373,31.962466,69.584857,1,647,9.0
5,2019,451,15.075388,24.018855,1,244,7.0
6,2018,191,10.753927,17.634809,1,149,5.0
7,2017,117,10.760684,19.121503,1,133,5.0
8,2016,65,7.830769,14.118438,1,72,4.0
9,2015,30,8.466667,12.378327,1,46,3.0


2020년 코로나 이후로 운영하던 에어비앤비를 접은 호스트가 꽤 있나?

### 12개월 리뷰수

In [74]:
london_lists.columns

Index(['숙소_id', '숙소_url', '호스트_id', '슈퍼호스트', '숙소_지역', '숙소_특징', '숙소_유형',
       '수용_인원수', '욕실수', '침실수', '침대수', '숙소_가격', '예약_가능여부', '리뷰수', '12개월_리뷰수',
       '30일_리뷰수', '마지막_리뷰', '리뷰점수', '숙소_정확성_리뷰점수', '숙소_청결도_리뷰점수',
       '숙소_체크인_리뷰점수', '숙소_소통_리뷰점수', '숙소_위치_리뷰점수', '숙소_가격_리뷰점수', '평균_리뷰수',
       '마지막_리뷰_year'],
      dtype='object')

In [75]:
# 12개월 리뷰수가 0인 조건 설정
condition_reviewforyear_0 = london_lists['12개월_리뷰수'] == 0

# 조회할 컬럼 지정 
col_lists = ['숙소_id', '숙소_유형', '숙소_가격', '리뷰수', '12개월_리뷰수', '마지막_리뷰', '리뷰점수']

# 출력 : 최근 1년 간 리뷰가 없던 데이터 개수 : 4226개
print(f'최근 1년 간 리뷰가 없던 데이터 개수: {london_lists[col_lists][condition_reviewforyear_0].shape[0]}개')

# 최근 1년간 리뷰가 없던 숙소를 리뷰수, 리뷰 점수 순으로 내림차순 정렬
london_lists[col_lists][condition_reviewforyear_0].sort_values(by=['리뷰수', '리뷰점수'], ascending=False)

최근 1년 간 리뷰가 없던 데이터 개수: 4226개


,숙소_id,숙소_유형,숙소_가격,리뷰수,12개월_리뷰수,마지막_리뷰,리뷰점수
780,604185,Private room,44.0,647,0,2020-08-01,4.94
885,877864,Private room,53.0,575,0,2020-03-15,4.95
1144,1296836,Private room,65.0,559,0,2020-10-26,4.89
13151,15855571,Private room,100.0,354,0,2020-03-05,4.91
622,615456,Entire home/apt,60.0,350,0,2021-05-17,4.63
...,...,...,...,...,...,...,...
52747,716795569980171182,Entire home/apt,315.0,1,0,2023-02-28,1.00
54474,751963048368738317,Private room,240.0,1,0,2022-11-27,1.00
56194,777303959471723427,Entire home/apt,254.0,1,0,2023-01-29,1.00
58049,812059225245832456,Private room,45.0,1,0,2023-03-01,1.00


In [76]:
# 최근 1년간 리뷰가 없던 숙소_가격, 리뷰수 기초 통계량 조회 

london_lists[condition_reviewforyear_0][['숙소_가격', '리뷰수']].describe()

,숙소_가격,리뷰수
count,4226.000000,4226.000000
mean,238.331519,14.586370
std,1238.264128,31.193729
min,1.000000,1.000000
25%,74.000000,2.000000
50%,130.000000,5.000000
75%,228.000000,14.000000
max,53588.000000,647.000000


In [77]:
# 비교를 위한 전체 데이터의 숙소_가격, 리뷰수 기초통계량 조회 
london_lists[['숙소_가격', '리뷰수']].describe()

,숙소_가격,리뷰수
count,42489.000000,42489.000000
mean,165.598908,28.924992
std,423.604661,54.148481
min,0.000000,1.000000
25%,71.000000,3.000000
50%,121.000000,11.000000
75%,195.000000,30.000000
max,53588.000000,1457.000000


In [78]:
# 1년 간 리뷰가 있던 데이터의 숙소_가격, 리뷰수 기초통계량 조회 
london_lists[~condition_reviewforyear_0][['숙소_가격', '리뷰수']].describe()

,숙소_가격,리뷰수
count,38263.000000,38263.000000
mean,157.565873,30.508638
std,171.172405,55.885939
min,0.000000,1.000000
25%,71.000000,4.000000
50%,120.000000,12.000000
75%,191.000000,32.000000
max,10000.000000,1457.000000


In [79]:
# '리뷰_여부' 컬럼 추가: '12개월_리뷰수'가 0이면 0, 1 이상이면 1
london_lists['12개월_리뷰_여부'] = london_lists['12개월_리뷰수'].apply(lambda x: 0 if x == 0 else 1)


# 마지막 리뷰 년도별 :  숙소 가격 기초 통계 요약
리뷰_12개월_여부_숙소가격 = \
london_lists.groupby('12개월_리뷰_여부')['숙소_가격'].agg(
    count='size',
    mean='mean',
    std='std',
    min='min',
    max='max',
    median='median'
).reset_index().sort_values(by='12개월_리뷰_여부', ascending=False).reset_index(drop=True)

리뷰_12개월_여부_숙소가격

,12개월_리뷰_여부,count,mean,std,min,max,median
0,1,38263,157.565873,171.172405,0.0,10000.0,120.0
1,0,4226,238.331519,1238.264128,1.0,53588.0,130.0


- 마지막 리뷰 : 임의로 마지막 리뷰 작성 n년을 잡는 것보다  
- '12개월_리뷰수' : 지난 1년간 리뷰가 있던 숙소를 기준으로 잡는게 논리적으로 타당하다고 판단 

In [80]:
# 전체 데이터 대비 지난 1년간 리뷰가 없던 데이터의 비율 확인 
print(f'전체 데이터 개수: {london_lists.shape[0]}개')
print(f'지난 1년 간 리뷰가 없던 숙소의 개수: {london_lists[condition_reviewforyear_0].shape[0]}개')
print(f'지난 1년간 리뷰가 없던 숙소의 비율: {round((london_lists[condition_reviewforyear_0].shape[0] / london_lists.shape[0]) * 100, 2)}%')
print()

# 데이터 제거 
london_lists = london_lists[~condition_reviewforyear_0]

# 파생변수 컬럼 정리 
london_lists.drop(columns=['마지막_리뷰_year', '12개월_리뷰_여부'], inplace=True)

# 데이터 제거 후 확인
print(f'제거 후 숙소의 개수: {london_lists.shape[0]}개')

전체 데이터 개수: 42489개
지난 1년 간 리뷰가 없던 숙소의 개수: 4226개
지난 1년간 리뷰가 없던 숙소의 비율: 9.95%

제거 후 숙소의 개수: 38263개


# EDA 탐색적 데이터 분석 
- 가격에 대한 결측치 처리는 클러스터링으로 접근하는게 맞지 않나? 라는 가설로 접근 예정
- 클러스터링 : 유사한 성격을 가진 개체를 묶어 그룹으로 구분하는 것 
    - 파티션 기반(Partition-based) 클러스터링 : k-means, k-median
    - 계층적(Hierarchical) 클러스터링 : Agglomerative, Divisive
    - 밀도 기반(density-based) 클러스터링 : DBSCAN

<br />

- 이외 수용 인원수의 경우
    - 백분위수 기반 이상치 처리를 하고 싶은데.. 
    - Inner Fence / Outer Fence도 참고

## 수용인원수 이상치 처리

### 이상치 처리 전 숙소 유형 별 수용 인원수 확인 

In [81]:
# 조건 아카이브
condition_room_privateroom = london_lists['숙소_유형'] == 'Private room'
condition_room_entirehomeapt = london_lists['숙소_유형'] == 'Entire home/apt'


# 숙소 유형별 '수용_인원수' 기초 통계 요약

숙소유형별_수용인원수 = \
london_lists.groupby('숙소_유형')['수용_인원수'].agg(
    count='size',
    mean='mean',
    std='std',
    min='min',
    max='max',
    median='median'
).reset_index().sort_values(by='숙소_유형', ascending=False).reset_index(drop=True)

숙소유형별_수용인원수

,숙소_유형,count,mean,std,min,max,median
0,Private room,12349,1.893433,0.891678,1,16,2.0
1,Entire home/apt,25914,4.094929,2.065688,1,16,4.0


### 'Entire home/apt' 수용_인원수 이상치 처리

In [82]:
# 'Entire home/apt' 수용_인원수 박스플랏 확인

import plotly.express as px

# 박스 플롯 생성
fig = px.box(london_lists, y=london_lists['수용_인원수'][condition_room_entirehomeapt], title='Box Plot of 수용인원수 of Entire home/apt')

# 크기 설정 (너비: 6인치, 높이: 12인치)
fig.update_layout(
    width=6*96,  # 1인치 = 96픽셀
    height=12*96
)

# 출력
fig.show()

In [83]:
# Inner Fence

# 조건 설정
condition_room_entirehomeapt = london_lists['숙소_유형'] == 'Entire home/apt'

# IQR 정의
Q1 = london_lists['수용_인원수'][condition_room_entirehomeapt].quantile(.25)
Q2 = london_lists['수용_인원수'][condition_room_entirehomeapt].quantile(.50)
Q3 = london_lists['수용_인원수'][condition_room_entirehomeapt].quantile(.75)
IQR = Q3 - Q1 

# 출력
print('Q1 = ', Q1)
print('Q2 = ', Q2)
print('Q3 = ', Q3)
print('IQR = ', IQR)
print()

# Outlier 조건 설정
condition_accommodates_entirehomeapt_upper_outliers = london_lists['수용_인원수'][condition_room_entirehomeapt] > (Q3 + IQR * 1.5)
# condition_accommodates_lower_outliers = london_lists['수용_인원수'][condition_room_entirehomeapt] < (Q1 - IQR * 1.5)

# Upper Outlier >> 변수로 정리
accommodates_entirehomeapt_upper_outliers = london_lists[condition_room_entirehomeapt][condition_accommodates_entirehomeapt_upper_outliers]


# 출력
print(accommodates_entirehomeapt_upper_outliers.sort_values(by='수용_인원수', ascending=False))
print()
print(f'entire home/apt 데이터 중 "수용_인원수" Upper_outlier 개수 : {accommodates_entirehomeapt_upper_outliers.shape[0]}개')
print(f'entire home/apt 데이터의 "수용_인원수" Upper_outlier 비율 : {round((accommodates_entirehomeapt_upper_outliers.shape[0] / london_lists.shape[0]) * 100, 2)}%')

Q1 =  2.0
Q2 =  4.0
Q3 =  5.0
IQR =  3.0

                     숙소_id                                            숙소_url  \
54055   742547847677727765   https://www.airbnb.com/rooms/742547847677727765   
28711             32100949             https://www.airbnb.com/rooms/32100949   
22117             23757469             https://www.airbnb.com/rooms/23757469   
57125   795722439348038055   https://www.airbnb.com/rooms/795722439348038055   
11498             13905660             https://www.airbnb.com/rooms/13905660   
...                    ...                                               ...   
59978   833231757227295456   https://www.airbnb.com/rooms/833231757227295456   
59864   830291791859731495   https://www.airbnb.com/rooms/830291791859731495   
32148             36822733             https://www.airbnb.com/rooms/36822733   
59707   831348935165747019   https://www.airbnb.com/rooms/831348935165747019   
90117  1110081089140235705  https://www.airbnb.com/rooms/1110081089140235705  

In [84]:
# Upper outlier 데이터의 인덱스 추출
entirehomeapt_upper_outlier_indices = accommodates_entirehomeapt_upper_outliers.index

# upper outlier 제거한 데이터프레임 생성
london_lists_cleaned = london_lists.drop(index=entirehomeapt_upper_outlier_indices)

# london_lists에 업데이트
london_lists = london_lists_cleaned

# 출력 
print(f'entire home/apt 데이터 중 "수용_인원수" Upper_outlier 제거 후 데이터 개수 : {london_lists.shape[0]}개')

entire home/apt 데이터 중 "수용_인원수" Upper_outlier 제거 후 데이터 개수 : 37716개


### 'Private Room' 수용_인원수 이상치 처리

In [85]:
# 'Private Room' 수용_인원수 박스플랏 확인

import plotly.express as px

# 박스 플롯 생성
fig = px.box(london_lists, y=london_lists['수용_인원수'][condition_room_privateroom], title='Box Plot of 수용인원수 of Private Room')

# 크기 설정 (너비: 6인치, 높이: 12인치)
fig.update_layout(
    width=6*96,  # 1인치 = 96픽셀
    height=12*96
)

# 출력
fig.show()

In [86]:
# Inner Fence / Private Room

# 조건 정의
condition_room_privateroom = london_lists['숙소_유형'] == 'Private room'

# IQR 정의
Q1 = london_lists['수용_인원수'][condition_room_privateroom].quantile(.25)
Q2 = london_lists['수용_인원수'][condition_room_privateroom].quantile(.50)
Q3 = london_lists['수용_인원수'][condition_room_privateroom].quantile(.75)
IQR = Q3 - Q1 

# 출력
print('Q1 = ', Q1)
print('Q2 = ', Q2)
print('Q3 = ', Q3)
print('IQR = ', IQR)
print()

# Outlier 조건 설정
condition_accommodates_privateroom_upper_outliers = london_lists['수용_인원수'][condition_room_privateroom] > (Q3 + IQR * 1.5)
# condition_accommodates_lower_outliers = london_lists['수용_인원수'][condition_room_entirehomeapt] < (Q1 - IQR * 1.5)

# Upper & Lower Outlier >> 변수로 정리
accommodates_privateroom_upper_outliers = london_lists[condition_room_privateroom][condition_accommodates_privateroom_upper_outliers]
# accommodates_lower_outliers = london_lists['수용_인원수'][condition_room_entirehomeapt][condition_accommodates_lower_outliers]

# 출력
print(accommodates_privateroom_upper_outliers.sort_values(by='수용_인원수', ascending=False))
print()
print(f'Private Room 데이터 중 "수용_인원수" Upper_outlier 개수 : {accommodates_privateroom_upper_outliers.shape[0]}개')
print(f'Private Room 데이터의 "수용_인원수" Upper_outlier 비율 : {round((accommodates_privateroom_upper_outliers.shape[0] / london_lists.shape[0]) * 100, 2)}%')

Q1 =  1.0
Q2 =  2.0
Q3 =  2.0
IQR =  1.0

                     숙소_id                                            숙소_url  \
4780               6742681              https://www.airbnb.com/rooms/6742681   
15094             17925370             https://www.airbnb.com/rooms/17925370   
46058   587051580777165921   https://www.airbnb.com/rooms/587051580777165921   
20906             22606388             https://www.airbnb.com/rooms/22606388   
79021  1023737653404459810  https://www.airbnb.com/rooms/1023737653404459810   
...                    ...                                               ...   
42451             51629713             https://www.airbnb.com/rooms/51629713   
42508             51749795             https://www.airbnb.com/rooms/51749795   
42514             51752058             https://www.airbnb.com/rooms/51752058   
42846             52290935             https://www.airbnb.com/rooms/52290935   
90568  1112524736745357245  https://www.airbnb.com/rooms/1112524736745357245  

In [87]:
# Upper outlier 데이터의 인덱스 추출
privateroom_upper_outlier_indices = accommodates_privateroom_upper_outliers.index

# upper outlier 제거한 데이터프레임 생성
london_lists_cleaned = london_lists.drop(index=privateroom_upper_outlier_indices)

# london_lists에 업데이트
london_lists = london_lists_cleaned

# 출력 
print(f'privateroom 데이터 중 "수용_인원수" Upper_outlier 제거 후 데이터 개수 : {london_lists.shape[0]}개')

privateroom 데이터 중 "수용_인원수" Upper_outlier 제거 후 데이터 개수 : 37011개


### 이상치 제거 후 수용 인원수 확인 

In [88]:
# 조건 아카이브
condition_room_privateroom = london_lists['숙소_유형'] == 'Private room'
condition_room_entirehomeapt = london_lists['숙소_유형'] == 'Entire home/apt'

# 숙소 유형별 '수용_인원수' 기초 통계 요약

숙소유형별_수용인원수_이상치제거 = \
london_lists.groupby('숙소_유형')['수용_인원수'].agg(
    count='size',
    mean='mean',
    std='std',
    min='min',
    max='max',
    median='median'
).reset_index().sort_values(by='숙소_유형', ascending=False).reset_index(drop=True)

숙소유형별_수용인원수_이상치제거

,숙소_유형,count,mean,std,min,max,median
0,Private room,11644,1.733854,0.554061,1,3,2.0
1,Entire home/apt,25367,3.933890,1.744865,1,9,4.0


## 숙소 가격 이상치 처리 

### 이상치 처리 전 숙소 유형별 숙소 가격 확인 

In [89]:
london_lists.columns

Index(['숙소_id', '숙소_url', '호스트_id', '슈퍼호스트', '숙소_지역', '숙소_특징', '숙소_유형',
       '수용_인원수', '욕실수', '침실수', '침대수', '숙소_가격', '예약_가능여부', '리뷰수', '12개월_리뷰수',
       '30일_리뷰수', '마지막_리뷰', '리뷰점수', '숙소_정확성_리뷰점수', '숙소_청결도_리뷰점수',
       '숙소_체크인_리뷰점수', '숙소_소통_리뷰점수', '숙소_위치_리뷰점수', '숙소_가격_리뷰점수', '평균_리뷰수'],
      dtype='object')

In [90]:
# 조건 아카이브
condition_room_privateroom = london_lists['숙소_유형'] == 'Private room'
condition_room_entirehomeapt = london_lists['숙소_유형'] == 'Entire home/apt'

# 숙소 유형별 '숙소_가격' 기초 통계 요약

숙소유형별_숙소가격 = \
london_lists.groupby('숙소_유형')['숙소_가격'].agg(
    count='size',
    mean='mean',
    std='std',
    min='min',
    max='max',
    median='median'
).reset_index().sort_values(by= 'count', ascending=False).reset_index(drop=True)

숙소유형별_숙소가격

,숙소_유형,count,mean,std,min,max,median
0,Entire home/apt,25367,192.436788,144.912814,13.0,4500.0,155.0
1,Private room,11644,68.956716,169.281796,0.0,10000.0,58.0


### 'Entire home/apt' 숙소_가격 이상치 처리

In [91]:
# 'Entire home/apt' 숙소_가격 박스플랏 확인

import plotly.express as px

# 박스 플롯 생성
fig = px.box(london_lists, y=london_lists['숙소_가격'][condition_room_entirehomeapt], title='Box Plot of 숙소 가격 of Entire home/apt')

# 크기 설정 (너비: 6인치, 높이: 12인치)
fig.update_layout(
    width=6*96,  # 1인치 = 96픽셀
    height=12*96
)

# 출력
fig.show()

In [92]:
# Inner Fence

# 조건 설정
condition_room_entirehomeapt = london_lists['숙소_유형'] == 'Entire home/apt'

# IQR 정의
Q1 = london_lists['숙소_가격'][condition_room_entirehomeapt].quantile(.25)
Q2 = london_lists['숙소_가격'][condition_room_entirehomeapt].quantile(.50)
Q3 = london_lists['숙소_가격'][condition_room_entirehomeapt].quantile(.75)
IQR = Q3 - Q1 

# 출력
print('Q1 = ', Q1)
print('Q2 = ', Q2)
print('Q3 = ', Q3)
print('IQR = ', IQR)
print()

# Outlier 조건 설정
condition_price_entirehomeapt_upper_outliers = london_lists['숙소_가격'][condition_room_entirehomeapt] > (Q3 + IQR * 1.5)
# condition_accommodates_lower_outliers = london_lists['숙소_가격'][condition_room_entirehomeapt] < (Q1 - IQR * 1.5)

# Upper Outlier >> 변수로 정리
price_entirehomeapt_upper_outliers_inner = london_lists[condition_room_entirehomeapt][condition_price_entirehomeapt_upper_outliers]


# 출력
print(price_entirehomeapt_upper_outliers_inner.sort_values(by='숙소_가격', ascending=False))
print()
print(f'entire home/apt 데이터 중 "숙소_가격" Upper_outlier 개수 : {price_entirehomeapt_upper_outliers_inner.shape[0]}개')
print(f'entire home/apt 데이터의 "숙소_가격" Upper_outlier 비율 : {round((price_entirehomeapt_upper_outliers_inner.shape[0] / london_lists.shape[0]) * 100, 2)}%')

Q1 =  114.0
Q2 =  155.0
Q3 =  225.0
IQR =  111.0

                     숙소_id                                            숙소_url  \
73510   973819591021406622   https://www.airbnb.com/rooms/973819591021406622   
27293             30254889             https://www.airbnb.com/rooms/30254889   
44620             54045342             https://www.airbnb.com/rooms/54045342   
72323   962858206000752165   https://www.airbnb.com/rooms/962858206000752165   
70312   946455317574084845   https://www.airbnb.com/rooms/946455317574084845   
...                    ...                                               ...   
79181  1025903077715437656  https://www.airbnb.com/rooms/1025903077715437656   
38074             44513629             https://www.airbnb.com/rooms/44513629   
15813             18468347             https://www.airbnb.com/rooms/18468347   
64581   893961410190605333   https://www.airbnb.com/rooms/893961410190605333   
59528   828882738325688921   https://www.airbnb.com/rooms/828882738325

In [93]:
# Outer Fence

# 조건 설정
condition_room_entirehomeapt = london_lists['숙소_유형'] == 'Entire home/apt'

# IQR 정의
Q1 = london_lists['숙소_가격'][condition_room_entirehomeapt].quantile(.25)
Q2 = london_lists['숙소_가격'][condition_room_entirehomeapt].quantile(.50)
Q3 = london_lists['숙소_가격'][condition_room_entirehomeapt].quantile(.75)
IQR = Q3 - Q1 

# 출력
print('Q1 = ', Q1)
print('Q2 = ', Q2)
print('Q3 = ', Q3)
print('IQR = ', IQR)
print()

# Outlier 조건 설정
condition_price_entirehomeapt_upper_outliers = london_lists['숙소_가격'][condition_room_entirehomeapt] > (Q3 + IQR * 3)
# condition_accommodates_lower_outliers = london_lists['숙소_가격'][condition_room_entirehomeapt] < (Q1 - IQR * 3)

# Upper Outlier >> 변수로 정리
price_entirehomeapt_upper_outliers_outer = london_lists[condition_room_entirehomeapt][condition_price_entirehomeapt_upper_outliers]


# 출력
print(price_entirehomeapt_upper_outliers_outer.sort_values(by='숙소_가격', ascending=False))
print()
print(f'entire home/apt 데이터 중 "숙소_가격" Upper_outlier 개수 : {price_entirehomeapt_upper_outliers_outer.shape[0]}개')
print(f'entire home/apt 데이터의 "숙소_가격" Upper_outlier 비율 : {round((price_entirehomeapt_upper_outliers_outer.shape[0] / london_lists.shape[0]) * 100, 2)}%')

Q1 =  114.0
Q2 =  155.0
Q3 =  225.0
IQR =  111.0

                    숙소_id                                           숙소_url  \
73510  973819591021406622  https://www.airbnb.com/rooms/973819591021406622   
27293            30254889            https://www.airbnb.com/rooms/30254889   
44620            54045342            https://www.airbnb.com/rooms/54045342   
72323  962858206000752165  https://www.airbnb.com/rooms/962858206000752165   
70312  946455317574084845  https://www.airbnb.com/rooms/946455317574084845   
...                   ...                                              ...   
76131  999021559021210605  https://www.airbnb.com/rooms/999021559021210605   
74775  986122502077726790  https://www.airbnb.com/rooms/986122502077726790   
42842            52197022            https://www.airbnb.com/rooms/52197022   
25715            28024155            https://www.airbnb.com/rooms/28024155   
25417            27775221            https://www.airbnb.com/rooms/27775221   

          호스트

#### 실제 데이터 조회_Outer Fence

In [94]:
# 조회할 컬럼 지정 
col_url = ['숙소_id', '숙소_url', '슈퍼호스트', '숙소_유형', '수용_인원수', '침대수', '숙소_가격','리뷰수', '12개월_리뷰수', '30일_리뷰수', '리뷰점수']

# 실제 데이터 조회_Out Fence
price_entirehomeapt_upper_outliers_outer[col_url].sort_values(by='숙소_가격')[:10]

,숙소_id,숙소_url,슈퍼호스트,숙소_유형,수용_인원수,침대수,숙소_가격,리뷰수,12개월_리뷰수,30일_리뷰수,리뷰점수
25417,27775221,https://www.airbnb.com/rooms/27775221,f,Entire home/apt,6,3.0,559.0,16,7,0,4.63
25715,28024155,https://www.airbnb.com/rooms/28024155,t,Entire home/apt,8,5.0,559.0,59,11,0,4.93
42842,52197022,https://www.airbnb.com/rooms/52197022,f,Entire home/apt,4,2.0,561.0,14,2,1,4.43
75456,993229564214021740,https://www.airbnb.com/rooms/993229564214021740,t,Entire home/apt,6,4.0,562.0,21,21,0,4.95
76131,999021559021210605,https://www.airbnb.com/rooms/999021559021210605,f,Entire home/apt,6,3.0,562.0,17,17,2,4.82
74775,986122502077726790,https://www.airbnb.com/rooms/986122502077726790,f,Entire home/apt,6,3.0,562.0,3,3,0,5.00
53393,730118413223051235,https://www.airbnb.com/rooms/730118413223051235,t,Entire home/apt,6,0.0,563.0,20,14,0,4.90
33229,38132777,https://www.airbnb.com/rooms/38132777,f,Entire home/apt,7,4.0,564.0,1,1,0,2.00
36312,41928513,https://www.airbnb.com/rooms/41928513,f,Entire home/apt,6,4.0,565.0,1,1,0,3.00
38974,45889624,https://www.airbnb.com/rooms/45889624,f,Entire home/apt,6,3.0,565.0,37,8,0,4.46


In [95]:
price_entirehomeapt_upper_outliers_outer[['수용_인원수', '침대수', '숙소_가격','리뷰수', '12개월_리뷰수', '30일_리뷰수']].describe()

,수용_인원수,침대수,숙소_가격,리뷰수,12개월_리뷰수,30일_리뷰수
count,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000
mean,5.983114,3.300188,839.881801,15.742964,6.557223,0.333959
std,1.629641,1.286110,404.850886,21.077452,7.959425,0.816368
min,1.000000,0.000000,559.000000,1.000000,1.000000,0.000000
25%,5.000000,2.000000,617.000000,2.000000,2.000000,0.000000
50%,6.000000,3.000000,726.000000,7.000000,4.000000,0.000000
75%,7.000000,4.000000,900.000000,20.000000,8.000000,0.000000
max,9.000000,9.000000,4500.000000,145.000000,60.000000,8.000000


#### 실제 데이터 조회_inner Fence

In [96]:
# 실제 데이터 조회_inner Fence
price_entirehomeapt_upper_outliers_inner[col_url].sort_values(by='숙소_가격')[:10]

,숙소_id,숙소_url,슈퍼호스트,숙소_유형,수용_인원수,침대수,숙소_가격,리뷰수,12개월_리뷰수,30일_리뷰수,리뷰점수
64581,893961410190605333,https://www.airbnb.com/rooms/893961410190605333,t,Entire home/apt,4,2.0,392.0,20,20,1,4.90
15813,18468347,https://www.airbnb.com/rooms/18468347,t,Entire home/apt,8,6.0,392.0,31,9,0,4.81
38074,44513629,https://www.airbnb.com/rooms/44513629,t,Entire home/apt,5,3.0,392.0,13,5,0,4.85
59528,828882738325688921,https://www.airbnb.com/rooms/828882738325688921,f,Entire home/apt,4,2.0,392.0,24,3,0,4.88
79181,1025903077715437656,https://www.airbnb.com/rooms/1025903077715437656,f,Entire home/apt,7,4.0,392.0,1,1,1,4.00
53774,737359833269596194,https://www.airbnb.com/rooms/737359833269596194,f,Entire home/apt,4,2.0,393.0,83,57,3,4.90
36301,41847915,https://www.airbnb.com/rooms/41847915,f,Entire home/apt,7,2.0,393.0,21,7,0,5.00
34839,40057502,https://www.airbnb.com/rooms/40057502,f,Entire home/apt,8,5.0,393.0,14,1,0,5.00
60464,843395134970641612,https://www.airbnb.com/rooms/843395134970641612,f,Entire home/apt,4,1.0,393.0,4,4,0,3.75
33313,38171113,https://www.airbnb.com/rooms/38171113,t,Entire home/apt,4,3.0,393.0,21,9,0,4.86


In [97]:
price_entirehomeapt_upper_outliers_inner[['수용_인원수', '침대수', '숙소_가격','리뷰수', '12개월_리뷰수', '30일_리뷰수']].describe()

,수용_인원수,침대수,숙소_가격,리뷰수,12개월_리뷰수,30일_리뷰수
count,1608.000000,1608.000000,1608.000000,1608.000000,1608.000000,1608.000000
mean,5.732587,3.148632,583.318408,18.142413,7.286070,0.392413
std,1.626823,1.285474,297.455976,25.403090,8.562389,0.902664
min,1.000000,0.000000,392.000000,1.000000,1.000000,0.000000
25%,4.000000,2.000000,428.000000,3.000000,2.000000,0.000000
50%,6.000000,3.000000,500.000000,9.000000,4.000000,0.000000
75%,7.000000,4.000000,614.250000,23.000000,10.000000,0.000000
max,9.000000,9.000000,4500.000000,247.000000,98.000000,8.000000


#### 숙소_가격 이상치 제거 

outfence로 지워보자 우선.

In [98]:
# Outer Fence

# 조건 설정
condition_room_entirehomeapt = london_lists['숙소_유형'] == 'Entire home/apt'

# IQR 정의
Q1 = london_lists['숙소_가격'][condition_room_entirehomeapt].quantile(.25)
Q2 = london_lists['숙소_가격'][condition_room_entirehomeapt].quantile(.50)
Q3 = london_lists['숙소_가격'][condition_room_entirehomeapt].quantile(.75)
IQR = Q3 - Q1 

# 출력
print('Q1 = ', Q1)
print('Q2 = ', Q2)
print('Q3 = ', Q3)
print('IQR = ', IQR)
print()

# Outlier 조건 설정
condition_price_entirehomeapt_upper_outliers = london_lists['숙소_가격'][condition_room_entirehomeapt] > (Q3 + IQR * 3)
# condition_accommodates_lower_outliers = london_lists['숙소_가격'][condition_room_entirehomeapt] < (Q1 - IQR * 3)

# Upper Outlier >> 변수로 정리
price_entirehomeapt_upper_outliers_outer = london_lists[condition_room_entirehomeapt][condition_price_entirehomeapt_upper_outliers]


# 출력
print(price_entirehomeapt_upper_outliers_outer.sort_values(by='숙소_가격', ascending=False))
print()
print(f'entire home/apt 데이터 중 "숙소_가격" Upper_outlier 개수 : {price_entirehomeapt_upper_outliers_outer.shape[0]}개')
print(f'entire home/apt 데이터의 "숙소_가격" Upper_outlier 비율 : {round((price_entirehomeapt_upper_outliers_outer.shape[0] / london_lists.shape[0]) * 100, 2)}%')

Q1 =  114.0
Q2 =  155.0
Q3 =  225.0
IQR =  111.0

                    숙소_id                                           숙소_url  \
73510  973819591021406622  https://www.airbnb.com/rooms/973819591021406622   
27293            30254889            https://www.airbnb.com/rooms/30254889   
44620            54045342            https://www.airbnb.com/rooms/54045342   
72323  962858206000752165  https://www.airbnb.com/rooms/962858206000752165   
70312  946455317574084845  https://www.airbnb.com/rooms/946455317574084845   
...                   ...                                              ...   
76131  999021559021210605  https://www.airbnb.com/rooms/999021559021210605   
74775  986122502077726790  https://www.airbnb.com/rooms/986122502077726790   
42842            52197022            https://www.airbnb.com/rooms/52197022   
25715            28024155            https://www.airbnb.com/rooms/28024155   
25417            27775221            https://www.airbnb.com/rooms/27775221   

          호스트

In [99]:
# Outer Upper outlier 데이터의 인덱스 추출
price_entirehomeapt_upper_outlier_indices = price_entirehomeapt_upper_outliers_outer.index

# Outer upper outlier 제거한 데이터프레임 생성
london_lists_cleaned = london_lists.drop(index=price_entirehomeapt_upper_outlier_indices)

# london_lists에 업데이트
london_lists = london_lists_cleaned

# 출력 
print(f'entire home/apt 데이터 중 "숙소_가격" Upper_outlier 제거 후 데이터 개수 : {london_lists.shape[0]}개')

entire home/apt 데이터 중 "수용_인원수" Upper_outlier 제거 후 데이터 개수 : 36478개


### Private room 가격 이상치 처리

In [101]:
# Outer Fence

# 조건 설정
condition_room_privateroom = london_lists['숙소_유형'] == 'Private room'

# IQR 정의
Q1 = london_lists['숙소_가격'][condition_room_privateroom].quantile(.25)
Q2 = london_lists['숙소_가격'][condition_room_privateroom].quantile(.50)
Q3 = london_lists['숙소_가격'][condition_room_privateroom].quantile(.75)
IQR = Q3 - Q1 

# 출력
print('Q1 = ', Q1)
print('Q2 = ', Q2)
print('Q3 = ', Q3)
print('IQR = ', IQR)
print()

# Outlier 조건 설정
condition_price_privateroom_upper_outliers = london_lists['숙소_가격'][condition_room_privateroom] > (Q3 + IQR * 3)

# Upper Outlier >> 변수로 정리
price_privateroom_upper_outliers_outer = london_lists[condition_room_privateroom][condition_price_privateroom_upper_outliers]


# 출력
print(price_privateroom_upper_outliers_outer.sort_values(by='숙소_가격', ascending=False))
print()
print(f'Priavate room 데이터 중 "숙소_가격" Upper_outlier 개수 : {price_privateroom_upper_outliers_outer.shape[0]}개')
print(f'Priavate room 데이터의 "숙소_가격" Upper_outlier 비율 : {round((price_privateroom_upper_outliers_outer.shape[0] / london_lists.shape[0]) * 100, 2)}%')

Q1 =  45.0
Q2 =  58.0
Q3 =  75.0
IQR =  30.0

                     숙소_id                                            숙소_url  \
80543  1036166747787748467  https://www.airbnb.com/rooms/1036166747787748467   
852                 698606               https://www.airbnb.com/rooms/698606   
33290             38157709             https://www.airbnb.com/rooms/38157709   
38282             44732052             https://www.airbnb.com/rooms/44732052   
38253             44696103             https://www.airbnb.com/rooms/44696103   
...                    ...                                               ...   
40503             49221377             https://www.airbnb.com/rooms/49221377   
28110             31457764             https://www.airbnb.com/rooms/31457764   
42739             51862116             https://www.airbnb.com/rooms/51862116   
6200               8143954              https://www.airbnb.com/rooms/8143954   
33384             38245616             https://www.airbnb.com/rooms/382456

In [102]:
# Outer Upper outlier 데이터의 인덱스 추출
price_privateroom_upper_outlier_indices = price_privateroom_upper_outliers_outer.index

# Outer upper outlier 제거한 데이터프레임 생성
london_lists_cleaned = london_lists.drop(index=price_privateroom_upper_outlier_indices)

# london_lists에 업데이트
london_lists = london_lists_cleaned

# 출력 
print(f'Private room 데이터 중 "숙소_가격" Upper_outlier 제거 후 데이터 개수 : {london_lists.shape[0]}개')

Private room 데이터 중 "숙소_가격" Upper_outlier 제거 후 데이터 개수 : 36268개


### 가격 = 0인 데이터 삭제 

In [105]:
# 조건 생성
condition_price_0 = london_lists['숙소_가격'] == 0

# 데이터 조회 
london_lists[col_url][condition_price_0]

,숙소_id,숙소_url,슈퍼호스트,숙소_유형,수용_인원수,침대수,숙소_가격,리뷰수,12개월_리뷰수,30일_리뷰수,리뷰점수
40878,49765360,https://www.airbnb.com/rooms/49765360,f,Private room,1,1.0,0.0,2,2,0,4.5


In [107]:
# 데이터 값 조회를 위한 조건 지정
id_49765360 = london_lists['숙소_id'] == 49765360

# 데이터 제거(Outlier 제거)
london_lists = london_lists[~id_49765360]

### 전처리 후 숙소가격 기초통계랑 확인

In [108]:
# 숙소 유형별 '숙소_가격' 기초 통계 요약

숙소유형별_숙소가격 = \
london_lists.groupby('숙소_유형')['숙소_가격'].agg(
    count='size',
    mean='mean',
    std='std',
    min='min',
    max='max',
    median='median'
).reset_index().sort_values(by= 'count', ascending=False).reset_index(drop=True)

숙소유형별_숙소가격

,숙소_유형,count,mean,std,min,max,median
0,Entire home/apt,24834,178.540992,93.535468,13.0,558.0,151.0
1,Private room,11433,61.762704,24.681922,8.0,165.0,57.0


## 상관관계 분석